# Portfolio Performance & Risk Analysis

## Table of Contents
- [1. Analysis of Portfolio Constituents](#section-1)
    * [1.1 Daily Returns](#subsection-11)
    * [1.2 Normalized Plot of Closing Prices](#subsection-12) 
    * [1.3 Histogram of Returns](#subsection-13) 
    * [1.4 Correlation Matrix](#subsection-14) 
    * [1.5 Normalized Plot of Cumulative Returns](#subsection-15)
    * [1.6 Box & Whisker Plot of Returns](#subsection-16) 
    * [1.7 Dollar Triangle](#subsection-17) 
    * [1.8 Risk vs Return Trade-off](#subsection-18) 
    * [1.9 Wins vs Losses](#subsection-19)
    * [1.10 Statistical Attributes of Returns](#subsection-110) 
    * [1.11 What About Drawdowns and Downside Risk?](#subsection-111) 
    * [1.12 Tail Risk](#subsection-112) 
    * [1.13 Price Crash Risk](#subsection-113)   
    * [1.14 Liquidity Risk](#subsection-114)     
$$$$
- [2. Construction of Portfolios](#section-2)
    * [2.1 Price-Weighted Portfolio (Monthly Rebalancing)](#subsection-21)
    * [2.2 Equally-Weighted Portfolio](#subsection-22)
    * [2.3 Market-Value-Weighted Portfolio (Monthly Rebalancing)](#subsection-23)
    * [2.4 Inverse Volatility Portfolio (Monthly Rebalancing)](#subsection-24)
    * [2.5 Random-Weighted Portfolios](#subsection-25)
    * [2.6 Global Minimum (Annualized) Variance Portfolio](#subsection-26)
    * [2.7 Maximum (Annualized) Sharpe Portfolio](#subsection-27)
    * [2.8 Variance Minimization using Scipy](#subsection-28)
    * [2.9 Sharpe Maximization using Scipy](#subsection-29)
    * [2.10 Portfolio Comparison : Performance against Benchmark](#subsection-210)
    * [2.11 Plotting the Efficient Frontier and the Capital Asset Line](#subsection-211)
$$$$
- [3. Putting It All Together](#section-3)
    * [Impact and Magnitude of Diversification](#subsection-31)
$$$$
- [4. Capital Asset Pricing Model](#section-4)
    * [4.1 Market Portfolio and Annualized Covariance](#subsection-41)
    * [4.2 Beta, Systematic Risk, and Idiosyncratic Risk](#subsection-42)
    * [4.3 Rolling Beta](#subsection-43)
    * [4.4 Expected Return and Alpha](#subsection-44)
    * [4.5 Security Market Line](#subsection-45)
$$$$
- [5. Farma-French 5-Factor Model](#section-5)
    * [5.1 Factor Data](#subsection-51)
    * [5.2 Rolling Correlation With Factors](#subsection-52)
    * [5.3 Linear Regression](#subsection-53)
    * [5.4 Regression Coefficients : Magnitude of Factors Driving Returns](#subsection-54)
    * [5.5 Model Evaluation and Residual Analysis](#subsection-55)

## Objective 

This notebook is an introductory guide to understanding stock returns from a statistical perspective, with a focus on the different types of underlying risk. Traditional portfolio construction methods are implemented to show the time-tested effectiveness of diversification in optimizing risk-adjusted returns. Theoretical models for forecasting returns showcase the importance of capturing each asset's relationship with the market (from a linear standpoint).

## Importing the Necessary Libraries 

In [ ]:
import pandas as pd 
import pandas_datareader.data as web
import numpy as np 
import yfinance as yf 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import optimize, stats  
from sklearn import linear_model, metrics 
from statsmodels import tools, regression 
from statsmodels.graphics import gofplots
from statsmodels.stats import stattools
from statsmodels.tsa.stattools import adfuller, kpss
from matplotlib import ticker 
np.random.seed(789)
# pd.set_option("display.max_rows", None)  
# pd.set_option("display.max_columns", None) 
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats("svg")

## Loading the Data 

In [ ]:
open_tech = pd.read_excel("file_path/Open.xlsx", parse_dates = ["Date"], index_col = "Date")

high_tech = pd.read_excel("file_path/High.xlsx", parse_dates = ["Date"], index_col = "Date")

low_tech = pd.read_excel("file_path/Low.xlsx", parse_dates = ["Date"], index_col = "Date")

close_tech = pd.read_excel("file_path/Close.xlsx", parse_dates = ["Date"], index_col = "Date")

adj_close_tech = pd.read_excel("file_path/Adj Close.xlsx", parse_dates = ["Date"], index_col = "Date")

shares_outstd_tech = pd.read_excel("file_path/Shares Outstanding.xlsx", parse_dates = ["Date"], index_col = "Date")

volume_tech = pd.read_excel("file_path/Volume.xlsx", parse_dates = ["Date"], index_col = "Date")

In [ ]:
rf_rate = 0.03
start_date = "2019-12-31"
end_date = "2024-01-01" 
tickers = close_tech.columns.tolist( )

## Key Assumptions 

- **Long-only, stocks-only portfolio for long-term investing**
> No short positions are allowed, and all positions are held for at least a month. Portfolios are constructed and rebalanced based on traditional allocation methods, and do not incorporate technical analysis or machine learning. 

- **Execution of portfolio rebalancing orders at the closing price of each month's first day (market order)**
> In reality, a large block order is broken down into smaller limit orders executed at different offer prices throughout the day. Other order types can be used, such as Immediate or Cancel, All or None, Fill or Kill, pegged, or reserve orders.

- **Absence of trading costs & rebates**
> Commissions and rebates are not taken into account. Calculations of effective cost and realized cost require the execution price and the Bid-Ask midpoint at the time of each smaller order. Without these, net returns cannot be computed.

- **Absence of dividends**
> The compounding effect of reinvesting dividends is not taken into account. Without this, total returns cannot be computed. 

- **Use of arithmetic mean return**
> Though less used, geometric mean return is more accurate by taking into account serial correlation and period over period compounding.

## 1. Analysis of Portfolio Constituents 
<a id="section-1"></a>

### 1.1 Daily Returns
<a id="subsection-11"></a>

$$
\text{Daily Return}_{t} = \frac{\text{Close Price}_{t} - \text{Close Price}_{t-1}}{\text{Close Price}_{t-1}}
$$

In [ ]:
returns = adj_close_tech.pct_change(periods = 1).iloc[1:] 

In [ ]:
returns.round(4)

### 1.2 Normalized Plot of Closing Prices
<a id="subsection-12"></a>

$$
\text{Normalized Close Price}_{t} = \frac{\text{Close Price}_{t}}{\text{Close Price}_{0}} \times 100
$$ 

In [ ]:
adj_close_tech = adj_close_tech.iloc[1:]
norm_close_tech = adj_close_tech.div(adj_close_tech.iloc[0]).mul(100)

In [ ]:
def plot_normalized_close(norm_close_df, freq = None) :
    """
    Arguments :
    - norm_close_df : DataFrame of normalized closing prices 
    - freq (optional) : str for resampling frequency -> weekly or monthly or quarterly or annual 

    Returns :
    - None 
    
    Function :
    - Draws a line plot of normalized closing prices for each asset
    """
    if freq : 
        resampling = {"weekly" : "W-Fri", "monthly" : "BM", "quarterly" : "BQ", "annual" : "A"} 
        norm_close_df = norm_close_df.resample(resampling[freq]).last( )
    
    plt.figure(figsize = (12, 7))
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
    
    assets = norm_close_df.columns
    for i, asset in enumerate(assets) :
        plt.plot(norm_close_df[asset], c = colors[i], label = asset)
    
    plt.xlabel(xlabel = "")
    plt.ylabel(ylabel = "Normalized Close ($)", fontsize = 12)
    plt.legend(loc = "best")
    plt.style.use("default") 
    plt.show( )   

In [ ]:
plot_normalized_close(norm_close_tech)

### 1.3 Histogram of Returns
<a id="subsection-13"></a>

In [ ]:
def plot_histogram(returns_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 

    Returns :
    - None 
    
    Function :
    - Draws a histogram and a kernel density estimation plot of daily returns for each asset 
    - Highlights the minimum return, average return, and maximum return with dotted vertical lines 
    """
    assets = returns_df.columns
    n_assets = len(returns_df.columns)

    if n_assets % 2 != 0 :
        n_rows = int(n_assets / 2) + 1
    else :
        n_rows = int(n_assets / 2)
    
    fig, ax = plt.subplots(n_rows, 2, figsize = (12, n_rows * 5))
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]

    for i in range(n_rows) :
        for j in range(2) :
            if i + i + j <= n_assets - 1 :
                asset = assets[i + i + j]
                min_ret = returns_df[asset].min( ) * 100
                mean_ret = returns_df[asset].mean( ) * 100
                max_ret = returns_df[asset].max( ) * 100
            
                ax[i , j].hist(x = returns_df[asset] * 100, bins = 50, density = True, label = asset, color = colors[i + i + j])
                sns.kdeplot(returns_df[asset] * 100, ax = ax[i , j], color = "black")
                ax[i , j].axvline(x = min_ret, color = "red", linestyle = "--", label = f"{round(min_ret, 2)}")
                ax[i , j].axvline(x = mean_ret, color = "gold", linestyle = "--", label = f"{round(mean_ret, 2)}")
                ax[i , j].axvline(x = max_ret, color = "green", linestyle = "--", label = f"{round(max_ret, 2)}")
                
                ax[i , j].set_xlabel(xlabel = "Daily Return (%)", fontsize = 12)
                if (i + i + j) % 2 == 0 :
                    ax[i , j].set_ylabel("Density", fontsize = 12)
                else :
                    ax[i , j].set_ylabel("")
                ax[i , j].legend(fontsize = 10)
                
    plt.style.use("default") 
    plt.show( )   

In [ ]:
plot_histogram(returns[tickers])

### 1.4 Correlation Matrix
<a id="subsection-14"></a>

$$
\text{r}_{X, Y} = \frac{\sum_{t=1}^{T} (\text{Return}_{X,t} - \bar{\text{Return}_X}) \cdot (\text{Return}_{Y,t} - \bar{\text{Return}_Y})}{\sqrt{\sum_{t=1}^{T} (\text{Return}_{X,t} - \bar{\text{Return}_X})^2 \cdot \sum_{t=1}^{T} (\text{Return}_{Y,t} - \bar{\text{Return}_Y})^2}}
$$ 

$$
\rho = 1 - \frac{6 \times \sum d_i^2}{T \times (T^2 - 1)} 
$$

In [ ]:
def plot_corr_heatmap(returns_df, method = "pearson") :
    """
    Arguments :
    - returns_df : DataFrame of daily returns
    - method : str for the method for computing correlation -> pearson or spearman or kendall

    Returns :
    - None 
    
    Function :
    - Computes the correlation matrix
    - Removes repetitive correlation coefficients between the assets
    - Applies a green colormap to display the strength of correlation 
    """
    corr_mat = returns_df.corr(method = method) 
    n_assets = len(corr_mat.columns)
    
    for i in range(n_assets - 1) :
        corr_mat.iloc[i , i + 1 :] = np.nan 
    
    plt.figure(figsize = (n_assets, n_assets))
    sns.heatmap(corr_mat, annot = True, square = True, vmin = 0.55, vmax = 0.85, cmap = "Greens", cbar = False)
    sns.set_theme(font_scale = 1) 
    
    plt.xlabel(xlabel = "")
    plt.ylabel(ylabel = "")
    plt.style.use("default") 
    plt.show( )   

In [ ]:
plot_corr_heatmap(returns[tickers])

In [ ]:
plot_corr_heatmap(returns[tickers], method = "spearman")

### 1.5 Normalized Plot of Cumulative Returns 
<a id="subsection-15"></a>

$$ 
\text{Cumulative Return}_{T} = \left( \prod_{t=1}^{T} 1 + \text{Return}_{t} \right) - 1
$$

In [ ]:
cum_returns_stocks = (1 + returns[tickers]).cumprod( ) - 1 

In [ ]:
cum_returns_stocks.round(4)

In [ ]:
def plot_cum_returns(cum_returns_df) :
    """
    Arguments :
    - cum_returns_df : DataFrame of cumulative returns 

    Returns :
    - None 
        
    Function :
    - Draws a line plot of cumulative daily returns for each asset 
    """
    plt.figure(figsize = (12, 7))
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
    
    assets = cum_returns_df.columns
    for i, asset in enumerate(assets) :
        plt.plot(cum_returns_df[asset].mul(100), c = colors[i], label = asset)
    
    plt.xlabel(xlabel = "")
    plt.ylabel(ylabel = "Cumulative Returns (%)", fontsize = 12)
    plt.legend(loc = "best")
    plt.style.use("default")
    plt.show( )  

In [ ]:
plot_cum_returns(cum_returns_stocks) 

### 1.6 Box & Whisker Plot of Returns 
<a id="subsection-16"></a>

$$ 
\text{Maximum} = \text{Q3} + 1.5 \times (\text{Q3} - \text{Q1}) 
$$ 

$$
\text{Q3} = \frac{4 \times (\text{N}+1)}{4}\text{-th ordered value}
$$ 

$$
\text{Median} = \frac{(\text{N}+1)}{2}\text{-th ordered value} 
$$

$$
\text{Q1} = \frac{(\text{N}+1)}{4}\text{-th ordered value}
$$ 

$$
\text{Minimum} = \text{Q1} - 1.5 \times (\text{Q3} - \text{Q1})
$$ 

In [ ]:
def plot_box_whisker(returns_df, freq = None) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    - freq (optional) : str for resampling frequency -> weekly or monthly or quarterly or annual 

    Returns :
    - None 
        
    Function :
    - Draws a box & whisker plot of returns for each asset (for the specified resampling frequency)
    - Plots outlier values of returns for each asset (for the specified resampling frequency)
    """
    n_assets = len(returns_df.columns)
    
    if freq : 
        resampling = {"weekly" : "W-Fri", "monthly" : "BM", "quarterly" : "BQ", "annual" : "A"} 
        returns_df = returns_df.resample(resampling[freq]).agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1) 
    
    plt.figure(figsize = (n_assets, 8))  
    sns.boxplot(data = returns_df.mul(100), palette = "tab10")

    plt.xlabel(xlabel = "")
    if freq :
        plt.ylabel(ylabel = f"{freq.title( )} Return (%)", fontsize = 12)
    else :
        plt.ylabel(ylabel = "Daily Return (%)", fontsize = 12)
    plt.style.use("default")
    plt.show( )   

In [ ]:
plot_box_whisker(returns[tickers])

In [ ]:
plot_box_whisker(returns[tickers], "weekly")

In [ ]:
plot_box_whisker(returns[tickers], "monthly")

In [ ]:
plot_box_whisker(returns[tickers], "quarterly")

In [ ]:
plot_box_whisker(returns[tickers], "annual")

### 1.7 Dollar Triangle 
<a id="subsection-17"></a>

In [ ]:
def dollar_triangle(returns_df, initial_balance = 100_000) : 
    """
    Arguments :
    - returns_df : DataFrame of daily returns for a particular asset  
    - initial_balance : int representing the initial investment in the asset (100_000 by default)
    
    Returns :
    - dollar_triangle_df 
    
    Function :
    - Resamples daily returns to annual frequency, and converts to annual log returns 
    - Computes the dollar value of the initial balance invested in the asset at the beginning of the year and held for n years 
    """
    ann_returns = returns_df.resample("A", kind = "period").agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1).to_frame( )
    ann_log_returns_df = np.log(ann_returns + 1).dropna( )
    
    max_years = ann_log_returns_df.index.size
    windows = list(range(max_years, 0, - 1)) 
    
    for n_years in windows :  
        ann_log_returns_df[f"{n_years} Y"] = np.exp(n_years * ann_log_returns_df.iloc[: , 0].rolling(n_years).mean( ))

    dollar_triangle_df = ann_log_returns_df.drop(columns = ann_log_returns_df.columns[0])
    dollar_triangle_df = dollar_triangle_df.mul(initial_balance)  
    return dollar_triangle_df 

In [ ]:
def plot_dollar_triangle(returns_df, initial_balance = 100_000) :  
    """
    Arguments :
    - returns_df : DataFrame of daily returns for a particular asset  
    - initial_balance : int representing the initial investment in the asset (100_000 by default)
    
    Returns :
    - None 
    
    Function :
    - Computes the dollar triangle for each asset 
    - Applies a green colormap to compare dollar values over time   
    """
    assets = returns_df.columns
    n_assets = len(returns_df.columns)

    if n_assets % 2 != 0 :
        n_rows = int(n_assets / 2) + 1
    else :
        n_rows = int(n_assets / 2)
    
    fig, ax = plt.subplots(n_rows, 2, figsize = (12, n_rows * 5))

    for i in range(n_rows) :
        for j in range(2) :
            if i + i + j <= n_assets - 1 :
                asset = assets[i + i + j]
                ticker_dollar_triangle = dollar_triangle(returns_df[asset], initial_balance = 100_000)
            
                sns.heatmap(data = ticker_dollar_triangle.T, annot = True, square = True, fmt = ".0f", cmap = "Greens", cbar = False, ax = ax[i , j])
            
                ax[i , j].tick_params(axis = "y", left = False, labelleft = False, labelright = True)
                ax[i , j].set_xlabel(xlabel = "Year End", fontsize = 12)
                ax[i , j].set_ylabel(ylabel = f"Holding Period in {asset}", fontsize = 12, rotation = 90, va = "center", labelpad = - 270)
    
    plt.style.use("default") 
    plt.show( )   

In [ ]:
plot_dollar_triangle(returns[tickers])

### 1.8 Risk vs Return Trade-off 
<a id="subsection-18"></a>

**Number of Trading Days** :
- Weekly : 5
- Monthly : 21
- Quarterly : 63
- Annual : 252

$$
\bar{\text{Return}}_{Frequency} = \frac{1}{T} \left( \sum_{t=1}^{T} \text{Return}_{t} \right) \times \text{Number of Trading Days}_{Frequency}  
$$ 

$$
\sigma_{Frequency} = \sqrt{\frac{1}{T}\sum_{t=1}^{T}(\text{Return}_{t} - \bar{\text{Return}})^2} \times \sqrt{\text{Number of Trading Days}_{Frequency}} 
$$ 

$$
\text{Sharpe Ratio}_{Frequency} = \frac{\bar{\text{Return}}_{Frequency}}{\sigma_{Frequency}}
$$ 

In [ ]:
def risk_return(returns_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - summary_df : DataFrame displaying the average return, standard deviation, and sharpe ratio for each resampling frequency
    
    Function :
    - Computes the average daily return and average daily standard deviation
    - Converts the average daily return and average daily standard deviation to weekly, monthly, quarterly, and annual frequency 
    """      
    summary_df = returns_df.agg(["mean" , "std"]).transpose( )
    summary_df.columns = ["Daily Return" , "Daily Volatility"] 
    summary_df["Daily Sharpe"] = (summary_df["Daily Return"]).div(summary_df["Daily Volatility"]) 
    
    n_trading_days = {"weekly" : 5, "monthly" : 21, "quarterly" : 63, "annual" : 252}
    for freq in n_trading_days.keys( ) :
        
        summary_df[f"{freq.title( )} Return"] = summary_df["Daily Return"] * n_trading_days[freq] 
        summary_df[f"{freq.title( )} Volatility"] =  summary_df["Daily Volatility"] * np.sqrt(n_trading_days[freq])  
        summary_df[f"{freq.title( )} Sharpe"] = (summary_df[f"{freq.title( )} Return"]).div(summary_df[f"{freq.title( )} Volatility"])
        
    summary_df.columns = pd.MultiIndex.from_product([["Daily", "Weekly", "Monthly", "Quarterly", "Annual"],
                                                     ["Return", "Volatility", "Sharpe"]])
    return summary_df 

In [ ]:
summary_stocks = risk_return(returns[tickers])

In [ ]:
summary_stocks.style\
              .format("{:.4f}")\
              .highlight_max(color = "lightgreen")\
              .highlight_min(color = "lightcoral")

In [ ]:
plt.figure(figsize = (13, 8))
plt.scatter(x = summary_stocks[("Annual", "Volatility")].mul(100), y = summary_stocks[("Annual", "Return")].mul(100), 
            s = 40, c = summary_stocks[("Annual", "Sharpe")], cmap = "RdYlGn")

for ticker in summary_stocks.index : 
    plt.annotate(ticker, xy = (summary_stocks.loc[ticker , ("Annual", "Volatility")] * 100 - 1,  summary_stocks.loc[ticker , ("Annual", "Return")] * 100 + 1), size = 10)

plt.xlabel(xlabel = "Annualized Volatility (%)", fontsize = 12)
plt.ylabel(ylabel = "Annualized Return (%)", fontsize = 12) 
plt.xticks(ticks = range(25, 71, 5))
plt.grid( )
plt.colorbar(label = "Annual Sharpe")
plt.style.use("default")
plt.show( )

### 1.9 Wins vs Losses
<a id="subsection-19"></a>

$$
\text{Best Return}_{Frequency} = \max (\text{Frequency Return}_{t = 1}, \text{Frequency Return}_{t = 2}, \text{Frequency Return}_{t = 3}, \ldots, \text{Frequency Return}_{t = T}) 
$$ 

$$
\text{Worst Return}_{Frequency} = \min (\text{Frequency Return}_{t = 1}, \text{Frequency Return}_{t = 2}, \text{Frequency Return}_{t = 3}, \ldots, \text{Frequency Return}_{t = T}) 
$$ 

In [ ]:
def best_worst_returns(returns_df):
    """
    Arguments:
    - returns_df: DataFrame of daily returns

    Returns:
    - best_worst_df: DataFrame displaying the best and worst returns for each resampling frequency

    Function:
    - Iteratively resamples daily returns
    - For each resampling frequency, computes the maximum return and minimum return for each asset
    """
    best_worst = { }

    resampling = {"D" : "Daily", "W-Fri": "Weekly", "BM": "Monthly", "BQ": "Quarterly", "A": "Annual"}
    for freq in resampling.keys( ):
        
        if freq == "D" :
            periodic_returns_df = returns_df.copy( )
        else :
            periodic_returns_df = returns_df.resample(freq).agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1)
        
        best_worst[f"Best {resampling[freq]} Return"] = np.array(periodic_returns_df.max( ))
        best_worst[f"Worst {resampling[freq]} Return"] = np.array(periodic_returns_df.min( ))

    best_worst_df = pd.DataFrame(data = best_worst, index = returns_df.columns)
    best_worst_df.columns = pd.MultiIndex.from_product([["Daily", "Weekly", "Monthly", "Quarterly", "Annual"],
                                                        ["Best Return", "Worst Return"]])
    return best_worst_df

In [ ]:
best_worst_stocks = best_worst_returns(returns[tickers])

In [ ]:
best_worst_stocks.style\
                 .format("{:.4f}")\
                 .highlight_max(color = "lightgreen")\
                 .highlight_min(color = "lightcoral")

$$
\text{Batting Average}_{Frequency} = \frac{\text{Number of Periods with Positive Returns}}{\text{T}} 
$$ 

In [ ]:
def win_percentage(returns_df) : 
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - win_df : DataFrame displaying the % of rows with positive returns and the gain-pain ratio for each resampling frequency 
    
    Function :
    - Iteratively resamples daily returns 
    - For each resampling frequency, divides the number of timestamps with return > 0 by the total number of timestamps 
    """
    win = { }

    resampling = {"D" : "Day", "W-Fri" : "Week", "BM" : "Month", "BQ" : "Quarter", "A" : "Year"}
    for freq in resampling.keys( ) :
        
        if freq == "D" :
            periodic_returns_df = returns_df.copy( )
        else :
            periodic_returns_df = returns_df.resample(freq).agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1)
        
        percent_win_list = [  ] 
        gain_pain_list = [  ]
        for asset in returns_df.columns :
            asset_periodic_returns = periodic_returns_df[asset].copy( )
            
            positive_returns = asset_periodic_returns.loc[asset_periodic_returns > 0]
            negative_returns = asset_periodic_returns.loc[asset_periodic_returns < 0] 
            
            percent_win_period = (len(positive_returns) / len(asset_periodic_returns)) * 100
            percent_win_list.append(percent_win_period)
            
        win[f"% of Winning {resampling[freq]}s"] = percent_win_list

    win_df = pd.DataFrame(data = win, index = returns_df.columns)
    return win_df   

In [ ]:
win_percent_stocks = win_percentage(returns[tickers])

In [ ]:
win_percent_stocks.round(2)

$$ 
\text{Gain-Pain Ratio}_{Frequency} = \frac{\sum_{t=1}^{T} \max(0, \text{Return}_{t})}{\text{|}\sum_{t=1}^{T} \min(0, \text{Return}_{t})\text{|}}
$$ 

$$ 
\text{Pay Off Ratio}_{Frequency} = \frac{\frac{1}{T}\sum_{t=1}^{T} \text{Return}_{t} > 0}{\text{|}\frac{1}{T}\sum_{t=1}^{T} \text{Return}_{t} < 0\text{|}}
$$ 

$$ 
\text{Kelly Criterion}_{Frequency} = \frac{\text{Pay Off Ratio}_{Frequency} \times \text{Batting Average}_{Frequency} - (\text{1 - Batting Average}_{Frequency})}{\text{Pay Off Ratio}_{Frequency}}
$$ 

In [ ]:
def win_loss_ratios(returns_df) :  
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - win_loss_df : DataFrame displaying the gain-pain ratio, payoff ratio, and kelly criterion for each resampling frequency 
    
    Function :
    - Iteratively resamples daily returns 
    - For each resampling frequency : 
      - Divides the sum of positive returns by the absolute value of the sum of negative returns 
      - Divides the average positive return by the absolute value of the average negative return 
    """
    win_loss = { }

    resampling = {"D" : "Daily", "W-Fri" : "Weekly", "BM" : "Monthly", "BQ" : "Quarterly", "A" : "Annual"}
    for freq in resampling.keys( ) :
        
        if freq == "D" :
            periodic_returns_df = returns_df.copy( )
        else :
            periodic_returns_df = returns_df.resample(freq).agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1)
        
        gain_pain_list = [  ]
        payoff_list = [  ]
        kelly_list = [  ]
        for asset in returns_df.columns :
            asset_periodic_returns = periodic_returns_df[asset].copy( )
            
            positive_returns = asset_periodic_returns.loc[asset_periodic_returns > 0]
            negative_returns = asset_periodic_returns.loc[asset_periodic_returns < 0] 
            
            gain_pain_period = positive_returns.sum( ) / np.abs(negative_returns.sum( ))
            gain_pain_list.append(gain_pain_period)
            
            payoff_period = positive_returns.mean( ) / np.abs(negative_returns.mean( ))
            payoff_list.append(payoff_period)

            percent_win_period = len(positive_returns) / len(asset_periodic_returns) 
            kelly_period = ((payoff_period * percent_win_period - (1 - percent_win_period)) / payoff_period) * 100
            kelly_list.append(kelly_period)
            
        win_loss[f"{resampling[freq]} Gain-Pain Ratio"] = gain_pain_list
        win_loss[f"{resampling[freq]} Pay Off Ratio"] = payoff_list
        win_loss[f"{resampling[freq]} Kelly Criterion (%)"] = kelly_list

    win_loss_df = pd.DataFrame(data = win_loss, index = returns_df.columns)
    win_loss_df.columns = pd.MultiIndex.from_product([["Daily", "Weekly", "Monthly", "Quarterly", "Annual"],
                                                      ["Gain-Pain Ratio", "Pay Off Ratio", "Kelly Criterion (%)"]]) 
    return win_loss_df   

In [ ]:
win_loss_stocks = win_loss_ratios(returns[tickers])

In [ ]:
win_loss_stocks.style\
               .format("{:.4f}")\
               .highlight_max(color = "lightgreen")\
               .highlight_min(color = "lightcoral")

In [ ]:
def streak(returns_df) : 
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - streak_df : DataFrame displaying the longest streak of days with positive / negative returns for each resampling frequency 
    
    Function :
    - Iteratively resamples daily returns 
    - For each resampling frequency, computes the highest number of consecutive days with returns > 0 / returns < 0
    """
    streak = { }

    resampling = {"D" : "Day", "W-Fri" : "Week", "BM" : "Month", "BQ" : "Quarter", "A" : "Year"}
    for freq in resampling.keys( ) :
        
        if freq == "D" :
            periodic_returns_df = returns_df.copy( )
        else :
            periodic_returns_df = returns_df.resample(freq).agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1)
        
        longest_win_list = [  ]
        longest_lose_list = [  ]
        for asset in returns_df.columns :
            asset_periodic_returns = periodic_returns_df[asset].copy( )
            
            longest_win_streak = 0
            current_win_streak = 0      
            for return_value in asset_periodic_returns.values :
                if return_value > 0 :
                    current_win_streak += 1
                    if current_win_streak > longest_win_streak :
                        longest_win_streak = current_win_streak
                else :
                    current_win_streak = 0
            longest_win_list.append(longest_win_streak)                   
            
            longest_lose_streak = 0
            current_lose_streak = 0      
            for return_value in asset_periodic_returns.values :
                if return_value < 0 :
                    current_lose_streak += 1
                    if current_lose_streak > longest_lose_streak :
                        longest_lose_streak = current_lose_streak
                else :
                    current_lose_streak = 0
            longest_lose_list.append(longest_lose_streak) 
            
        streak[f"Longest Streak of Winning {resampling[freq]}s"] = longest_win_list
        streak[f"Longest Streak of Losing {resampling[freq]}s"] = longest_lose_list
    
    streak_df = pd.DataFrame(data = streak, index = returns_df.columns)   
    streak_df = pd.concat(axis = 1, objs = [ streak_df.loc[: , streak_df.columns.str.contains("Winning")] , 
                                             streak_df.loc[: , streak_df.columns.str.contains("Losing")] ])
    streak_df.columns = pd.MultiIndex.from_product([["Longest Winning Streak", "Longest Losing Streak"],
                                                    ["Days", "Weeks", "Months", "Quarters", "Years"]]) 
    return streak_df   

In [ ]:
win_lose_streak_stocks = streak(returns[tickers])

In [ ]:
win_lose_streak_stocks

### 1.10 Statistical Attributes of Returns 
<a id="subsection-110"></a>

***Tests of Normality :***

$$ 
W = \frac{{(\sum_{i=1}^{T} a_i x_{(i)})^2}}{{\sum_{i=1}^{T} (x_i - \bar{x})^2}} 
$$ 

$\text{Where :}$
- $ T \text{ is the sample size.} $
- $ x_{(i)} \text{ is the \( i \)-th order statistic (i.e., the \( i \)-th smallest value in the sample).} $
- $ \bar{x} \text{ is the sample mean.} $
- $ a_i \text{ are constants obtained from the covariance matrix of the order statistics of a normal sample.} $

The Shapiro-Wilk test evaluates the null hypothesis that the sample is normally distributed. If the p-value associated with the test statistic $W$ is less than a chosen significance level (e.g., 0.05), the null hypothesis is rejected, suggesting that the sample does not follow a normal distribution.

$$
JB = \frac{T}{6} \left( S^2 + \frac{1}{4}(K-3)^2 \right)
$$

Where:
- $ T \text{ is the sample size.} $
- $ S \text{ is the sample skewness.} $
- $ K \text{ is the sample kurtosis.} $

The Jarque-Bera test evaluates the null hypothesis that the sample is normally distributed. If the p-value associated with the test statistic $JB$ is less than a chosen significance level (e.g., 0.05), the null hypothesis is rejected, suggesting that the sample does not follow a normal distribution.

$$ 
\text{ADF} = \frac{{\text{Coefficient Estimate} - 1}}{{\text{Standard Error}}} 
$$

Where:
- $ \text{Coefficient Estimate is the estimated coefficient of the lagged differenced series in the regression equation.} $
- $ \text{Standard error is the standard error of the coefficient estimate.} $

The Augmented Dickey-Fuller test evaluates the null hypothesis that the time series is stationary around a deterministic trend. If the p-value associated with the test statistic $ADF$ is less than a chosen significance level (e.g., 0.05), the null hypothesis is rejected. The time series has a unit root, indicating non-stationarity (or a stochastic trend).

$$
\text{KPSS} = \frac{{\sum_{t=1}^{T} (S_t - S_{t-1})^2 / T}}{{(\sum_{t=1}^{T} S_t^2 / T)}}
$$

Where:
- $ S_t \text{ is the cumulative sum of the deviations of the time series from its mean at time \( t \).} $
- $ T \text{ is the number of observations in the time series.} $

The Kwiatkowski-Phillips-Schmidt-Shin test evaluates the null hypothesis that the time series is stationary around a deterministic trend. If the p-value associated with the test statistic $KPSS$ is less than a chosen significance level (e.g., 0.05), the null hypothesis is rejected. The time series has a unit root, indicating non-stationarity (or a stochastic trend).

***Finding the Best-Fit Distribution :***

$$ 
D = \max_{i} | F(x_i) - S(x_i) | 
$$ 

$\text{Where :}$
- $ F(x) \text{ is the empirical cumulative distribution function (CDF) of the sample.} $
- $ S(x) \text{ is the theoretical CDF of the specified distribution.} $ 
- $ x_i \text{ are the observed data points.} $

The Kolmogorov-Smirnov test evaluates the null hypothesis that the sample is drawn from the specified distribution. The closer the p-value associated with the test statistic D is to 1, the more the null hypothesis is rejected (assuming p-value > significance level), suggesting that the sample follows more strongly the specified distribution. 

***Common Distributions for Fitting Stock Returns :***

$$ 
F(x) = \frac{1}{2} + \frac{1}{2} \cdot \text{sign}(x - \mu) \cdot \left[1 - \exp\left(-\left(\frac{|x - \mu|}{\beta}\right)^{\alpha}\right)\right] 
$$

$\text{Where :}$
- $ F(x) \text{ is the cumulative distribution function at \( x \)}. $
- $ \mu \text{ is the location parameter.} $
- $ \beta \text{ is the scale parameter.} $
- $ \alpha \text{ is the shape parameter.} $
- $ \text{sign}(x - \mu) \text{ is the sign function, which is 1 if \( x > \mu \), -1 if \( x < \mu \), and 0 if \( x = \mu \).} $
- $ |x - \mu| \text{ represents the absolute difference between \( x \) and \( \mu \).} $

The CDF defines the probability that a random variable following the **Generalized Normal Distribution** is less than or equal to a given value $x$.

$$
F(x) = \frac{1}{\sqrt{2 \pi}} \int_{-\infty}^{x} \frac{1}{\sigma \sqrt{1 + \left(\frac{y - \mu}{\gamma}\right)^2}} e^{-\frac{1}{2} \left(\frac{u}{\sigma \sqrt{1 + \left(\frac{y - \mu}{\gamma}\right)^2}}\right)^2} \, du 
$$

$\text{Where :}$
- $ F(x) \text{ is the cumulative distribution function at \( x \)}. $
- $ \mu \text{ is the location parameter.} $
- $ \gamma \text{ is the shape parameter (scale parameter in some formulations).} $
- $ \sigma \text{ is the scale parameter (shape parameter in some formulations).} $
- $ \text{e is the base of natural logarithms.} $
- $ \pi \text{ is the mathematical constant pi.} $

The CDF defines the probability that a random variable following the **Johnson SU Distribution** is less than or equal to a given value $x$.

$$ 
F(x) = \int_{-\infty}^{x} \frac{\Gamma\left(\frac{\nu + 1}{2}\right)}{\sqrt{\nu \pi} \Gamma\left(\frac{\nu}{2}\right)} \left(1 + \frac{t^2}{\nu}\right)^{-\frac{\nu + 1}{2}} \, dt 
$$

$\text{Where :}$
- $ F(x) \text{ is the cumulative distribution function at \( x \)} $
- $ \nu \text{ is the degrees of freedom parameter, which affects the shape and tail behavior of the distribution.} $
- $ \Gamma(\cdot) \text{ is the gamma function.} $
- $ t \text{ represents the random variable following the Student's t-distribution.} $

The CDF defines the probability that a random variable following the **Student's t Distribution** is less than or equal to a given value $x$.

In [ ]:
def distribution_attributes(returns_df) : 
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - distribution_df : DataFrame displaying ratios for analyzing the distribution of daily returns 
    
    Function :
    - Conducts the Shapiro-Wilik and Jarque-Bera tests of normality and computes the p-values 
    - Conducts the Augmented Dickey-Fuller, Kwiatkowski-Phillips-Schmidt-Shin tests of stationarity and computes the p-values 
    - Returns the distribution (and its p-value) that best fits each asset's daily returns amongst 20 different distributions 
    """   
    sw_list = [  ]
    jb_list = [  ]
    adf_list = [  ]
    kpss_list = [  ]
    best_dist_list = [  ]
    best_p_val_list = [  ]
    for asset in returns_df.columns : 
        
        p_val_sw = stats.shapiro(returns_df[asset])[1] 
        p_val_jb = stats.jarque_bera(returns_df[asset])[1]
        p_val_adf = adfuller(returns_df[asset])[1]
        p_val_kpss = kpss(returns_df[asset])[1]
        
        sw_list.append(p_val_sw)
        jb_list.append(p_val_jb)
        adf_list.append(p_val_adf)
        kpss_list.append(p_val_kpss)
        
        dist_names = ["lognorm", "foldnorm", "halfnorm", "gennorm", "exponnorm", "powernorm", "skewnorm", "truncnorm", "johnsonsb", "johnsonsu", 
                      "beta", "burr", "cauchy", "expon", "gamma", "genextreme", "invgauss", "laplace", "pareto", "t"]
        dist_results = [ ]
        for dist_name in dist_names:
            
            dist = getattr(stats, dist_name)
            params = dist.fit(returns_df[asset])
            stat, p_val = stats.kstest(returns_df[asset], dist_name, args = params)
            dist_results.append((dist_name, p_val))
        
        best_dist, best_p_val = (max(dist_results, key = lambda item: item[1]))
        best_dist_list.append(best_dist)
        best_p_val_list.append(best_p_val)
        
    tests_df = pd.DataFrame({"Shapiro-Wilk P-Value" : sw_list, "Jarque-Bera P-Value" : jb_list, 
                             "Augmented Dickey-Fuller P-Value" : adf_list, "Kwiatkowski-Phillips-Schmidt-Shin P-Value" : kpss_list, 
                             "Best-Fit Distribution" : best_dist_list, "Best-Fit P-Value" : best_p_val_list}, 
                            index = returns_df.columns)
    return tests_df 

In [ ]:
dist_attributes_stocks = distribution_attributes(returns[tickers])

In [ ]:
dist_attributes_stocks.round(2)

In [ ]:
def plot_qq(returns_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 

    Returns :
    - None 
    
    Function :
    - Draws a quantile-quantile plot of daily returns for each asset
    """
    assets = returns_df.columns
    n_assets = len(returns_df.columns)
    
    if n_assets % 2 != 0 :
        n_rows = int(n_assets / 2) + 1
    else :
        n_rows = int(n_assets / 2)
    
    fig, ax = plt.subplots(n_rows, 2, figsize = (12, n_rows * 5))

    for i in range(n_rows) :
        for j in range(2) :
            if i + i + j <= n_assets - 1 :
                asset = assets[i + i + j]
                gofplots.qqplot(returns_df[asset] * 100, line = "q", ax = ax[i , j])
                                
                ax[i , j].set_xlabel("Standard Deviations Away From Mean", fontsize = 12)
                if (i + i + j) % 2 == 0 :
                    ax[i , j].set_ylabel("Daily Returns (%)", fontsize = 12)
                else :
                    ax[i , j].set_ylabel("")
                ax[i , j].legend([asset], fontsize = 10)
    
    plt.style.use("default")
    plt.show( )

In [ ]:
plot_qq(returns[tickers])

### 1.11 What About Drawdowns and Downside Risk? 
<a id="subsection-111"></a>

$$ 
\text{Annualized Downside } \sigma = \sqrt{\frac{1}{T} \sum_{t=1}^{T} \min\left((\text{Return}_{t} - \bar{\text{Return}})^2, 0\right)} \times \sqrt{252} 
$$ 

$$
\text{Average Drawdown} = \frac{1}{T} \sum_{t=1}^{T} \text{Drawdown}_{t}
$$ 

$$
\text{Average Drawdown Duration} = \frac{\text{Number of Drawdown Days}}{\text{Number of Drawdowns}} 
$$ 

$$
\text{Maximum Drawdown} = \max (\text{Drawdown}_{t = 1}, \text{Drawdown}_{t = 2}, \text{Drawdown}_{t = 3}, \ldots, \text{Drawdown}_{t = T}) 
$$ 

$$
\text{Longest Drawdown Duration} = \max(\text{Drawdown Duration}_1, \text{Drawdown Duration}_2, \ldots, \text{Drawdown Duration}_N)
$$ 

$$
\text{Sterling Ratio} = \frac{\text{Annualized Return} - \text{Risk-Free Rate}}{\text{|} \frac{1}{N} \sum_{i=1}^{N} \text{Largest Drawdown}_i\text{|}} 
$$

$$
\text{Ulcer Index} = \sqrt{\frac{1}{T} \sum_{t=1}^{T} (\text{Drawdown}_{t})^2} \times 100
$$

$$
\text{Calmar Ratio} = \frac{\text{Annualized } \bar{\text{Return}}}{\text{|Maximum Drawdown|}}
$$ 

$$
\text{Sortino Ratio} = \frac{\text{Annualized } \bar{\text{Return}} - \text{Risk-Free Rate}}{\text{Annualized Downside } \sigma}
$$ 

$$
\text{Kappa Ratio} = \frac{\text{Annualized } \bar{\text{Return}} - \text{Risk-Free Rate}}{\text{Annualized Downside } \sigma + \text{|Skew|} + \text{|Kurtosis|}}
$$ 

In [ ]:
def downside(returns_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - downside_df : DataFrame displaying ratios related to downside risk  
    
    Function :
    - Computes the annualized semi-deviation, average drawdown, maximum drawdown, sterling ratio, ulcer index, calmar ratio, sortino ratio, and kappa ratio 
    """
    wealth_index = (1 + returns_df).cumprod( )
    previous_peaks = wealth_index.cummax( )
    drawdowns_df = (wealth_index - previous_peaks) / previous_peaks
    
    avg_drawdown = np.array(drawdowns_df.mean( )) 
    max_drawdown = np.array(drawdowns_df.min( )) 
    
    longest_dd_duration_list = [  ]
    avg_dd_duration_list = [  ]
    sterling_list = [  ]
    for asset in drawdowns_df.columns :
        
        longest_dd_streak = 0
        current_dd_streak = 0 
        n_drawdowns = 0
        for drawdown_value in drawdowns_df[asset].values :
            if drawdown_value < 0 :
                current_dd_streak += 1
                if current_dd_streak > longest_dd_streak :
                    longest_dd_streak = current_dd_streak
            else :
                n_drawdowns += 1 
                current_dd_streak = 0
        longest_dd_duration_list.append(longest_dd_streak)   
        
        n_dd_days = len(drawdowns_df[asset].loc[drawdowns_df[asset] < 0]) 
        avg_dd_duration = n_dd_days / n_drawdowns 
        avg_dd_duration_list.append(avg_dd_duration)
        
        ann_return = returns_df[asset].mean( ) * 252 
        worst_five_dds = drawdowns_df[asset].nsmallest(5).values
        avg_worst_dd = worst_five_dds.mean( )
        sterling = (ann_return - rf_rate) / np.abs(avg_worst_dd) 
        sterling_list.append(sterling)
        
    mean_squared_drawdowns = np.array(drawdowns_df.pow(2).mean( )) 
    ulcer_index = np.sqrt(mean_squared_drawdowns) * 100 
    
    avg_return = np.array(returns_df.mean( ).mul(252)) 
    calmar = avg_return / np.abs(max_drawdown) 
    
    downside_std_list = [  ]
    sortino_list = [  ]
    kappa_list = [  ]
    for asset in returns_df.columns : 
        
        downside_returns = returns_df[asset].loc[returns_df[asset] < 0]
        downside_std = downside_returns.std( ) * np.sqrt(252)
        downside_std_list.append(downside_std)
        
        ann_return = returns_df[asset].mean( ) * 252 
        sortino = (ann_return - rf_rate) / downside_std 
        sortino_list.append(sortino)
        
        skew = returns_df[asset].skew( )
        kurt = returns_df[asset].kurtosis( )
        kappa = (ann_return - rf_rate) / (downside_std + np.abs(skew) + np.abs(kurt)) 
        kappa_list.append(kappa)
        
    downside_df = pd.DataFrame(data = {"Annual Downside Standard Deviation" : downside_std_list, 
                                       "Average Drawdown" : avg_drawdown, "Average Drawdown Duration (Days)" : avg_dd_duration_list, 
                                       "Maximum Drawdown" : max_drawdown, "Longest Drawdown Duration (Days)" : longest_dd_duration_list, 
                                       "Sterling Ratio" : sterling_list, "Ulcer Index" : ulcer_index, 
                                       "Calmar Ratio" : calmar, "Sortino Ratio" : sortino_list, "Kappa Ratio" : kappa_list}, 
                               index = returns_df.columns)
    return downside_df 

In [ ]:
downside_stocks = downside(returns[tickers]) 

In [ ]:
downside_stocks.style\
               .format("{:.4f}")\
               .highlight_max(color = "lightgreen")\
               .highlight_min(color = "lightcoral")

$$
\text{Drawdown}_{t} = \min \left( 0 , \text{Cumulative Return}_{t} - \text{Previous Peak}_{t} \right) 
$$ 

In [ ]:
def plot_drawdowns(returns_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - None

    Function : 
    - Computes drawdowns for each asset
    - Plots area charts of drawdowns for each asset 
    """
    wealth_index = (1 + returns_df).cumprod( )
    previous_peaks = wealth_index.cummax( )
    drawdowns_df = (wealth_index - previous_peaks) / previous_peaks 
    
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
    
    ax = drawdowns_df.mul(100).plot(kind = "area", alpha = 0.5, figsize = (12 , len(returns_df.columns) * 3), fontsize = 10, 
                                    subplots = True, sharex = False, color = colors, xlabel = "", ylabel = "Drawdown (%)") 
    avg_drawdowns = np.array(drawdowns_df.mean( )) * 100
    
    for i in range(len(ax)) :
        ax[i].axhline(y = avg_drawdowns[i], color = "red", linestyle = "--", label = str(avg_drawdowns[i].round(2)))
        ax[i].legend( )
        
    plt.style.use("default") 
    plt.show( ) 

In [ ]:
plot_drawdowns(returns[tickers])

### 1.12 Tail Risk
<a id="subsection-112"></a>

$$ 
\text{Skew} = \frac{\frac{1}{T} \sum_{t=1}^{T} (\text{Return}_{t} - \bar{\text{Return}})^3}{\left(\frac{1}{T} \sum_{t=1}^{T} (\text{Return}_{t} - \bar{\text{Return}})^2\right)^{3/2}}
$$ 

$$
\text{Kurtosis} = \frac{\frac{1}{T} \sum_{t=1}^{T} (\text{Return}_{t} - \bar{\text{Return}})^4}{\left(\frac{1}{T} \sum_{t=1}^{T} (\text{Return}_{t} - \bar{\text{Return}})^2\right)^{2}}
$$

$$
\text{Average Lower Tail Return} = \frac{1}{T} \sum_{t=1}^{T} \text{(Return < % Threshold)}_{t}
$$

$$
\text{Average Upper Tail Return} = \frac{1}{T} \sum_{t=1}^{T} \text{(Return > (1 - %) Threshold)}_{t}
$$

$$
\text{Tail Ratio} = \frac{\text{Average Upper Tail Return}}{\text{|Average Lower Tail Return|}}
$$

$$
\text{Common Sense Ratio} = \text{Gain-Pain Ratio} \times \text{Tail Ratio}
$$

$$
\text{Outlier Win Ratio} = \frac{\frac{1}{T} \sum_{t=1}^{T} (\text{Return} > \text{99th percentile})_{t}}{\frac{1}{T} \sum_{t=1}^{T} (\text{Return > 0})_{t}}
$$

$$
\text{Outlier Loss Ratio} = \frac{\frac{1}{T} \sum_{t=1}^{T} (\text{Return} < \text{1st percentile})_{t}}{\frac{1}{T} \sum_{t=1}^{T} (\text{Return < 0})_{t}}
$$

In [ ]:
def tail_attributes(returns_df, threshold = 0.05) : 
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    - threshold : float representing the quantile for lower and upper tail returns -> 0.01 or 0.05 or 0.1 
    
    Returns :
    - tail_df : DataFrame displaying ratios for analyzing the tail characteristics of daily returns 
    
    Function :
    - Computes the skew, kurtosis, average lower tail return, average upper tail return, tail ratio, common sense ratio, outlier win ratio, and outlier loss ratio 
    """
    tail_df = returns_df.agg(["skew" , "kurt"]).transpose( )
    tail_df.columns = ["Skew" , "Kurtosis"]  
    
    avg_lt_return_list = [  ]
    avg_ut_return_list = [  ]
    tail_list = [  ]
    common_sense_list = [  ]
    outlier_win_list = [  ]
    outlier_loss_list = [  ]
    for asset in returns_df.columns :
        
        lower_tail_threshold = returns_df[asset].quantile(threshold)  
        lower_tail_returns = returns_df[asset].loc[returns_df[asset] < lower_tail_threshold]
        avg_lt_return = lower_tail_returns.mean( )
        
        upper_tail_threshold = returns_df[asset].quantile(1 - threshold)
        upper_tail_returns = returns_df[asset].loc[returns_df[asset] > upper_tail_threshold]
        avg_ut_return = upper_tail_returns.mean( )
        
        tail = avg_ut_return / np.abs(avg_lt_return)
        
        positive_returns = returns_df[asset].loc[returns_df[asset] > 0]
        negative_returns = returns_df[asset].loc[returns_df[asset] < 0] 
        gain_pain = positive_returns.sum( ) / np.abs(negative_returns.sum( ))
        common_sense = gain_pain * tail 
        
        avg_upper_outlier_return = returns_df[asset].quantile(0.99).mean( )
        avg_positive_return = returns_df[asset].loc[returns_df[asset] > 0].mean( )
        outlier_win = avg_upper_outlier_return / avg_positive_return
        
        avg_lower_outlier_return = returns_df[asset].quantile(0.01).mean( )
        avg_negative_return = returns_df[asset].loc[returns_df[asset] < 0].mean( )
        outlier_loss = avg_lower_outlier_return / avg_negative_return
        
        avg_lt_return_list.append(avg_lt_return)
        avg_ut_return_list.append(avg_ut_return)
        tail_list.append(tail)
        common_sense_list.append(common_sense)
        outlier_win_list.append(outlier_win)
        outlier_loss_list.append(outlier_loss)
    
    tail_df[f"Average Lower Tail Return (< {int(threshold * 100)}%)"] = avg_lt_return_list 
    tail_df[f"Average Upper Tail Return (> {int((1 - threshold) * 100)}%)"] = avg_ut_return_list 
    tail_df["Tail Ratio"] = tail_list
    tail_df["Common Sense Ratio"] = common_sense_list
    tail_df["Outlier Win Ratio (> 99%)"] = outlier_win_list
    tail_df["Outlier Loss Ratio (< 1%)"] = outlier_loss_list 

    return tail_df 

In [ ]:
tail_stocks = tail_attributes(returns[tickers], threshold = 0.05)

In [ ]:
tail_stocks.style\
           .format("{:.4f}")\
           .highlight_max(color = "lightgreen")\
           .highlight_min(color = "lightcoral")

$$
\text{Historic Value at Risk}_{\alpha} = \text{Quantile}_{\alpha}(\text{Historical Returns})
$$ 

$$
\text{Parametric Value at Risk}_{\alpha} = \mu + z_{\alpha} \times \sigma
$$ 

$$
\text{Cornish-Fisher Value at Risk}_{\alpha} = \mu + \sigma \times \left( z_{\alpha} + \frac{1}{6} (z_{\alpha}^2 - 1) \times \text{skewness} + \frac{1}{24}(z_{\alpha}^3 - 3z_{\alpha}) \times \text{kurtosis} - \frac{1}{36}(2z_{\alpha}^3 - 5z_{\alpha}) \times \text{skewness}^2 \right)
$$

$$
\text{Conditional Value at Risk}_{\alpha} = \frac{1}{1-\alpha} \int_{-\infty}^{\text{VaR}_{\alpha}} x \cdot f(x) \,dx
$$ 

In [ ]:
def Value_at_Risk(returns_df, significance = 0.05) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    - significance : float representing the significance level -> 0.01 or 0.05 or 0.1  
    
    Returns :
    - VaR_df : DataFrame displaying ratios related to Value at Risk 
    
    Function :
    - Computes the historic VaR, parametric VaR, cornish-fisher VaR, and Conditional VaR at the specified confidence level 
    """
    hist_VaR_list = list(np.percentile(returns_df, significance * 100, axis = 0)) 
    
    param_VaR_list = [  ]
    cf_VaR_list = [  ]
    for asset in returns_df.columns :
        
        avg_return = returns_df[asset].mean( )
        vol = returns_df[asset].std( )
        param_VaR = stats.norm.ppf(significance, avg_return, vol)
        param_VaR_list.append(param_VaR)
        
        skew = returns_df[asset].skew( )
        kurtosis = returns_df[asset].kurt( )
        z_score = stats.norm.ppf(significance)
        z_score_cf = z_score + (z_score**2 - 1) * skew / 6 + (z_score**3 - 3 * z_score) * kurtosis / 24 - (2 * z_score**3 - 5 * z_score) * (skew**2) / 36
        cf_VaR = avg_return + z_score_cf * vol
        cf_VaR_list.append(cf_VaR)
        
    cond_VaR_list = [  ]
    for i in range(len(returns_df.columns)) :
        asset = returns_df.columns[i]
        hist_VaR = hist_VaR_list[i]
        cond_VaR = returns_df[asset].loc[returns_df[asset] <= hist_VaR].mean( )
        cond_VaR_list.append(cond_VaR)
    
    confidence_level = int((1 - significance) * 100)
    VaR_df = pd.DataFrame({f"Historic Value at Risk ({confidence_level}%)" : hist_VaR_list, 
                           f"Parametric Value at Risk ({confidence_level}%)" : param_VaR_list, 
                           f"Cornish-Fisher Value at Risk ({confidence_level}%)" : cf_VaR_list, 
                           f"Conditional Value at Risk ({confidence_level}%)" : cond_VaR_list}, 
                           index = returns_df.columns) 
    return VaR_df 

In [ ]:
VaR_stocks = Value_at_Risk(returns[tickers])

In [ ]:
VaR_stocks.style\
          .format("{:.4f}")\
          .highlight_max(color = "lightgreen")\
          .highlight_min(color = "lightcoral")

In [ ]:
def plot_Value_at_Risk(VaR_df) :
    """
    Arguments :
    - VaR_df : DataFrame displaying ratios related to Value at Risk 

    Returns :
    - None 
    
    Function :
    - Plots a bar chart of historic, parametric, cornish-fisher, and conditional Value at Risk for each asset 
    """
    ax = VaR_df.mul(100).plot(kind = "bar", figsize = (12 , 20), fontsize = 10, rot = 0, subplots = True, sharex = False, width = 0.5, 
                              xlabel = "", ylabel = "%", title = ["", "", "", ""]) 
    for i in range(len(ax)) :
        ax[i].tick_params(axis = "x", bottom = False, top = True, labelbottom = False, labeltop = True)
        ax[i].axhline(VaR_df.iloc[: , i].mean( ) * 100, linestyle = "--", c = "red")
        
    plt.style.use("default")
    plt.show( ) 

In [ ]:
plot_Value_at_Risk(VaR_stocks) 

In [ ]:
def monte_carlo_simulation(returns_df, n_simulations = 10_000, n_days = 252) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    - n_simulations : int for the number of random walks to simulate (10_000 by default)
    - n_days : int for the number of forward days to forecast (252 by default)
    
    Returns :
    - None 
    
    Function :
    - Computes Monte Carlo simulations for the future price path of each asset (10_000 by default)
    - Plots the 5 simulations with the lowest ending value for each asset 
    """
    assets = returns_df.columns
    n_assets = len(returns_df.columns)
    
    if n_assets % 2 != 0 :
        n_rows = int(n_assets / 2) + 1
    else :
        n_rows = int(n_assets / 2)
    
    fig, ax = plt.subplots(n_rows, 2, figsize = (12, n_rows * 5))
    
    for i in range(n_rows) :
        for j in range(2) :
            if i + i + j <= n_assets - 1 :
                asset = assets[i + i + j]
                avg_return = returns_df[asset].mean( )
                vol = returns_df[asset].std( )
                
                if asset in tickers :
                    value_start = close_tech.iloc[-1, close_tech.columns.get_loc(asset)]
                else :
                    value_start = 10_000
                
                simulated_values = { }
                for s in range(n_simulations) :
                    rand_returns = np.random.normal(avg_return, vol, n_days) + 1
                    forecasted_values = value_start * (rand_returns).cumprod( )
                    simulated_values[s] = forecasted_values
                mc_df = pd.DataFrame(simulated_values)
                
                bottom_10_paths = mc_df.iloc[-1].nsmallest(5).index.tolist( )
                
                ax[i , j].plot(list(range(n_days)), mc_df[bottom_10_paths])
                ax[i , j].set_xlabel("Number of Forward Days", fontsize = 12)
                if (i + i + j) % 2 == 0 :
                    ax[i , j].set_ylabel("Asset Value ($)", fontsize = 12)
                else :
                    ax[i , j].set_ylabel("")
                ax[i , j].legend([asset], fontsize = 10)
                
    plt.style.use("default")
    plt.show( )

In [ ]:
monte_carlo_simulation(returns[tickers])

$$
f(x; \xi, \mu, \sigma)_{GEV} = \frac{1}{\sigma} \left[1 + \xi \left(\frac{x - \mu}{\sigma}\right)\right]^{-1/\xi - 1} \exp \left\{-\left[1 + \xi \left(\frac{x - \mu}{\sigma}\right)\right]^{-1/\xi}\right\}
$$ 

$\text{Where :}$
- $ x \text{ is the random variable (daily returns).} $
- $ \xi \text{ is the shape.} $
- $ \mu \text{ is the location (central tendency).} $ 
- $ \sigma \text{ is the scale parameter (spread).} $ 

In [ ]:
def plot_extreme_value_theory(returns_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - None 
    
    Function :
    - Fits a Generalized Extreme Value distribution to the daily returns of each asset 
    - Plots the GEV probability density curve for each asset 
    """
    assets = returns_df.columns
    n_assets = len(returns_df.columns)
    
    if n_assets % 2 != 0 :
        n_rows = int(n_assets / 2) + 1
    else :
        n_rows = int(n_assets / 2)
    
    fig, ax = plt.subplots(n_rows, 2, figsize = (12, n_rows * 5))
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
    
    for i in range(n_rows) :
        for j in range(2) :
            if i + i + j <= n_assets - 1 :
                asset = assets[i + i + j]
                
                shape, loc, scale = stats.genextreme.fit(returns_df[asset])
                x = np.linspace(stats.genextreme.ppf(0.001, shape, loc = loc, scale = scale),
                                stats.genextreme.ppf(0.999, shape, loc = loc, scale = scale), 100)
                gev_pdf = stats.genextreme.pdf(x, shape, loc = loc, scale = scale)
                
                ax[i , j].plot(x, gev_pdf, color = colors[i + i + j])
                
                ax[i , j].set_xlabel("Daily Returns", fontsize = 12)
                if (i + i + j) % 2 == 0 :
                    ax[i , j].set_ylabel("Probability Density", fontsize = 12)
                else :
                    ax[i , j].set_ylabel("")
                legend_label = f"{asset}\nξ = {shape:.2f}\nμ = {loc:.2f}\nσ = {scale:.2f}"
                ax[i , j].legend([legend_label], fontsize = 10)
                
    plt.style.use("default")
    plt.show( )

In [ ]:
plot_extreme_value_theory(returns[tickers])

### 1.13 Price Crash Risk 
<a id="subsection-113"></a>

$$
\text{Annual Down-to-Up } \sigma = \log \left(\frac{(n_u - 1) \times \sum_{t=1}^{T} min(0, \text{Weekly Return}_t)^2}{(n_d - 1) \times \sum_{t=1}^{T} max(0, \text{Weekly Return}_t)^2}\right)
$$

$$
\text{Annual Negative Coefficient of Skewness} = -\frac{n \times (n - 1)^{3/2} \times \sum_{t=1}^{T} \text{Weekly Return}_t^3}{(n - 1) \times (n - 2) \times \left(\sum_{t=1}^{T} \text{Weekly Return}_t^2\right)^{3/2}}
$$

In [ ]:
def crash_risk(returns_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    
    Returns :
    - crash_risk_df : DataFrame displaying ratios related to the risk of price crash
    
    Function :
    - For each year, computes the down-to-up volatility and negative coefficient of skewness 
    """
    weekly_returns = returns_df.resample("W-Fri").agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1) 
    
    annual_duvol_list = [  ]
    annual_ncskew_list = [  ]
    
    for asset in weekly_returns.columns :
        asset_returns = weekly_returns[asset].copy( )
        
        for year in weekly_returns.index.year.unique( ) :
            returns_in_year = asset_returns.loc[str(year)].copy( )
            mean_return = returns_in_year.mean( )
            
            up_returns = returns_in_year.loc[returns_in_year > mean_return]
            down_returns = returns_in_year.loc[returns_in_year < mean_return]
            n_up_weeks = len(up_returns)
            n_down_weeks = len(down_returns)
            sum_squared_up_returns = up_returns.pow(2).sum( )
            sum_squared_down_returns = down_returns.pow(2).sum( )
            
            annual_duvol = np.log(((n_down_weeks - 1) * sum_squared_down_returns) / ((n_up_weeks - 1) * sum_squared_up_returns))
            annual_duvol_list.append(annual_duvol)
            
            n_weeks = len(returns_in_year)
            sum_squared_returns = returns_in_year.pow(2).sum( )
            sum_cubed_returns = returns_in_year.pow(3).sum( )
            
            annual_ncskew = - ((n_weeks) * (n_weeks - 1)**(3/2) * sum_cubed_returns) / ((n_weeks - 1) * (n_weeks - 2) * (sum_squared_returns)**(3/2))
            annual_ncskew_list.append(annual_ncskew)
    
    multi_index = pd.MultiIndex.from_product([weekly_returns.columns, weekly_returns.index.year.unique( )], names = ["Ticker", "Year"])
    crash_risk_df = pd.DataFrame({"Down-to-Up Volatility" : annual_duvol_list, "Negative Coefficient of Skewness" : annual_ncskew_list}, 
                                 index = multi_index)
    crash_risk_df = crash_risk_df.unstack(level = "Year")
    return crash_risk_df 

In [ ]:
crash_stocks = crash_risk(returns[tickers])

In [ ]:
crash_stocks.style\
            .format("{:.4f}")\
            .highlight_max(color = "lightgreen")

### 1.14 Liquidity Risk 
<a id="subsection-114"></a>

## 2. Construction of Portfolios 
<a id="section-2"></a>

In [ ]:
def monthly_weights_rebalancing(weights_df) :
    """
    Arguments :
    - weights_df : DataFrame of weights of each asset at each timestamp 
    
    Returns :
    - daily_weights : DataFrame of daily weights of each asset  
        
    Function :
    - Sets the daily weights of each asset in each month equal to the weights of that month's first day 
    """
    daily_weights = weights_df.copy( ) 
    daily_weights["Year"] = daily_weights.index.year 
    daily_weights["Month"] = daily_weights.index.month 
    daily_weights = daily_weights.reset_index( )
    
    month_start_weights = daily_weights.groupby(["Year", "Month"]).first( ).reset_index(drop = True)
    daily_weights.set_index("Date", inplace = True)
    daily_weights.drop(columns = ["Year", "Month"], inplace = True)
    month_start_weights.set_index("Date", inplace = True)
    
    daily_weights.loc[~ (daily_weights.index.isin(month_start_weights.index)) ] = np.nan 
    daily_weights.ffill(inplace = True)  
    
    return daily_weights 

In [ ]:
def plot_weights_returns(weights_df, returns_df, benchmarks = ["^GSPC", "^NDXT", "IXN"]) :
    """
    Arguments :
    - weights_df : DataFrame of weights of each asset at each timestamp 
    - returns_df : DataFrame of daily returns 
    - benchmarks : list of str representing the benchmarks' tickers 
    
    Returns :
    - None 
        
    Function :
    - Plots cumulative returns of the portfolio and benchmark indices 
    - Plots the annualized 3-month and 6-month rolling volatility of the portfolio 
    - Plots the weights of each asset over time -> price-weighted / market-value-weighted 
    - Plots the fixed weights of each asset -> equally-weighted / minimum variance / maximum sharpe 
    """
    fig , ax = plt.subplots(figsize = (12 , 24), sharex = False, sharey = False, nrows = 3, ncols = 1)
    
    for benchmark in benchmarks :
        compare = yf.download(tickers = benchmark, start = start_date, end = end_date)
        close_compare = compare["Adj Close"].copy( ) 
        returns_compare = close_compare.pct_change(periods = 1).iloc[1:] 
        returns_df[benchmark] = returns_compare
    
    cum_returns = (1 + returns_df).cumprod( ) - 1 
    cum_returns.mul(100).plot(fontsize = 10, ax = ax[0])
    ax[0].set_xlabel(xlabel = "")
    ax[0].set_ylabel(ylabel = "Cumulative Returns (%)", fontsize = 12) 
    returns_df.drop(columns = benchmarks, inplace = True)
    
    monthly_returns = returns_df.resample("BM").agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1)
    monthly_returns["3 Months"] = monthly_returns.iloc[: , 0].rolling(3).std( ) * np.sqrt(12)
    monthly_returns["6 Months"] = monthly_returns.iloc[: , 0].rolling(6).std( ) * np.sqrt(12)
    
    ax[1].plot(monthly_returns["3 Months"] * 100, label = "3 Months", c = "red")
    ax[1].plot(monthly_returns["6 Months"] * 100, label = "6 Months", c = "fuchsia")
    ax[1].set_xlabel(xlabel = "")
    ax[1].set_ylabel(ylabel = "Annualized Rolling Volatility (%)", fontsize = 12)
    ax[1].legend( )
    
    asset = returns_df.columns[0]
    if asset == "EWP" or "MVP" in asset or "MSP" in asset :
        weights_series = weights_df.iloc[0]
        weights_series.name = "Weights"
        
        contains_zero = weights_series.round(8).isin([0]).any( )
        if contains_zero : 
            weights_series = weights_series.loc[weights_series.round(8) != 0]
        
        colors = sns.color_palette("Set3")
        ax[2].pie(weights_series, labels = weights_series.index, colors = colors, autopct = "%.2f%%", radius = 1.2)
        
    else :
        colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
        for i, asset in enumerate(weights_df.columns) :
            ax[2].plot(weights_df[asset].mul(100), color = colors[i], label = asset)

        ax[2].set_xlabel(xlabel = "")
        ax[2].set_ylabel(ylabel = "Monthly Weights (%)", fontsize = 12)
        ax[2].legend( )
    
    plt.style.use("default")
    plt.show( )  

### 2.1 Price-Weighted Portfolio (Monthly Rebalancing) 
<a id="subsection-21"></a>

$$
\text{Weight}_t = \frac{\text{Price}_t}{\sum_{i=1}^{N} \text{Price}_{i,t}}
$$ 

$$
\text{PWP Return}_t = \sum_{i=1}^{N} \text{Return}_{i,t} \times \text{Weight}_{i,t}
$$ 

In [ ]:
weights_PWP = monthly_weights_rebalancing( adj_close_tech.div(adj_close_tech.sum(axis = 1), axis = 0) )
returns["PWP"] = returns[tickers].mul(weights_PWP).sum(axis = 1)  

In [ ]:
summary_PWP = risk_return(returns["PWP"].to_frame( )) 
summary_PWP.round(4)

In [ ]:
best_worst_PWP = best_worst_returns(returns["PWP"].to_frame( ))
best_worst_PWP.round(4) 

In [ ]:
win_percent_PWP = win_percentage(returns["PWP"].to_frame( ))
win_percent_PWP.round(2)

In [ ]:
win_loss_PWP = win_loss_ratios(returns["PWP"].to_frame( )) 
win_loss_PWP.round(4)

In [ ]:
win_lose_streak_PWP = streak(returns["PWP"].to_frame( ))
win_lose_streak_PWP 

In [ ]:
dist_attributes_PWP = distribution_attributes(returns["PWP"].to_frame( ))
dist_attributes_PWP.round(4)

In [ ]:
downside_PWP = downside(returns["PWP"].to_frame( )) 
downside_PWP.round(4) 

In [ ]:
tail_PWP = tail_attributes(returns["PWP"].to_frame( ), threshold = 0.05)
tail_PWP.round(4) 

In [ ]:
VaR_PWP = Value_at_Risk(returns["PWP"].to_frame( ))
VaR_PWP.round(4)

In [ ]:
crash_PWP = crash_risk(returns["PWP"].to_frame( ))
crash_PWP.round(4)

In [ ]:
plot_weights_returns(weights_PWP, returns["PWP"].to_frame( )) 

### 2.2 Equally-Weighted Portfolio 
<a id="subsection-22"></a>

$$
\text{W}_t = \frac{1}{N}
$$ 

$$
\text{EWP Return}_t = \frac{1}{N} \times \sum_{i=1}^{N} \text{Return}_{i,t}
$$ 

In [ ]:
weights_EWP = pd.DataFrame(np.full_like(returns[tickers].values, 1 / len(tickers)), columns = tickers, index = returns.index)
returns["EWP"] = returns[tickers].mul(weights_EWP).sum(axis = 1)  

In [ ]:
summary_EWP = risk_return(returns["EWP"].to_frame( ))
summary_EWP.round(4)  

In [ ]:
best_worst_EWP = best_worst_returns(returns["EWP"].to_frame( ))
best_worst_EWP.round(4)  

In [ ]:
win_percent_EWP = win_percentage(returns["EWP"].to_frame( )) 
win_percent_EWP.round(2)

In [ ]:
win_loss_EWP = win_loss_ratios(returns["EWP"].to_frame( )) 
win_loss_EWP.round(4)

In [ ]:
win_lose_streak_EWP = streak(returns["EWP"].to_frame( ))
win_lose_streak_EWP 

In [ ]:
dist_attributes_EWP = distribution_attributes(returns["EWP"].to_frame( ))
dist_attributes_EWP.round(4) 

In [ ]:
downside_EWP = downside(returns["EWP"].to_frame( )) 
downside_EWP.round(4) 

In [ ]:
tail_EWP = tail_attributes(returns["EWP"].to_frame( ), threshold = 0.05)
tail_EWP.round(4) 

In [ ]:
VaR_EWP = Value_at_Risk(returns["EWP"].to_frame( ))
VaR_EWP.round(4) 

In [ ]:
crash_EWP = crash_risk(returns["EWP"].to_frame( ))
crash_EWP.round(4)

In [ ]:
plot_weights_returns(weights_EWP, returns["EWP"].to_frame( ))

### 2.3 Market-Value-Weighted Portfolio (Monthly Rebalancing) 
<a id="subsection-23"></a>

$$
\text{Weight}_t = \frac{\text{Price}_t \times \text{Shares Oustanding}_t}{\sum_{i=1}^{N} \text{Price}_{i,t} \times \text{Shares Outstanding}_{i,t}}
$$ 

$$
\text{MVWP Return}_t = \sum_{i=1}^{N} \text{Return}_{i,t} \times \text{Weight}_{i,t}
$$ 

In [ ]:
mkt_cap = shares_outstd_tech.iloc[1:].mul(close_tech.iloc[1:]) 
mkt_cap["MVWP"] = mkt_cap.sum(axis = 1)

weights_MVWP = monthly_weights_rebalancing( mkt_cap[tickers].div(mkt_cap["MVWP"], axis = 0) ) 
returns["MVWP"] = returns[tickers].mul(weights_MVWP).sum(axis = 1) 

In [ ]:
summary_MVWP = risk_return(returns["MVWP"].to_frame( ))
summary_MVWP.round(4)

In [ ]:
best_worst_MVWP = best_worst_returns(returns["MVWP"].to_frame( ))
best_worst_MVWP.round(4) 

In [ ]:
win_percent_MVWP = win_percentage(returns["MVWP"].to_frame( )) 
win_percent_MVWP.round(2)

In [ ]:
win_loss_MVWP = win_loss_ratios(returns["MVWP"].to_frame( )) 
win_loss_MVWP.round(4)

In [ ]:
win_lose_streak_MVWP = streak(returns["MVWP"].to_frame( ))
win_lose_streak_MVWP 

In [ ]:
dist_attributes_MVWP = distribution_attributes(returns["MVWP"].to_frame( ))
dist_attributes_MVWP.round(4) 

In [ ]:
downside_MVWP = downside(returns["MVWP"].to_frame( )) 
downside_MVWP.round(4) 

In [ ]:
tail_MVWP = tail_attributes(returns["MVWP"].to_frame( ), threshold = 0.05)
tail_MVWP.round(4) 

In [ ]:
VaR_MVWP = Value_at_Risk(returns["MVWP"].to_frame( ))
VaR_MVWP.round(4) 

In [ ]:
crash_MVWP = crash_risk(returns["MVWP"].to_frame( ))
crash_MVWP.round(4)

In [ ]:
plot_weights_returns(weights_MVWP, returns["MVWP"].to_frame( )) 

### 2.4 Inverse Volatility Portfolio (Monthly Rebalancing) 
<a id="subsection-24"></a>

$$
\text{Weight}_t = \frac{\frac{1}{\sigma_t}}{\sum_{i=1}^{N} \frac{1}{\sigma_{i, t}}}
$$ 

$$
\text{IVP Return}_t = \sum_{i=1}^{N} \text{Return}_{i,t} \times \text{Weight}_{i,t}
$$ 

In [ ]:
rolling_vol = returns[tickers].rolling(window = 21).std( )
inverse_vol = 1 / rolling_vol

weights_IVP = monthly_weights_rebalancing( inverse_vol.div(inverse_vol.sum(axis = 1), axis = 0) )
returns["IVP"] = returns[tickers].mul(weights_IVP).sum(axis = 1)  

In [ ]:
summary_IVP = risk_return(returns["IVP"].to_frame( ))
summary_IVP.round(4)

In [ ]:
best_worst_IVP = best_worst_returns(returns["IVP"].to_frame( ))
best_worst_IVP.round(4)

In [ ]:
win_percent_IVP = win_percentage(returns["IVP"].to_frame( )) 
win_percent_IVP.round(4)

In [ ]:
win_loss_IVP = win_loss_ratios(returns["IVP"].to_frame( )) 
win_loss_IVP.round(4)

In [ ]:
win_lose_streak_IVP = streak(returns["IVP"].to_frame( ))
win_lose_streak_IVP

In [ ]:
dist_attributes_IVP = distribution_attributes(returns["IVP"].to_frame( ))
dist_attributes_IVP.round(4) 

In [ ]:
downside_IVP = downside(returns["IVP"].to_frame( )) 
downside_IVP.round(4) 

In [ ]:
tail_IVP = tail_attributes(returns["IVP"].to_frame( ), threshold = 0.05)
tail_IVP.round(4) 

In [ ]:
VaR_IVP = Value_at_Risk(returns["IVP"].to_frame( ))
VaR_IVP.round(4) 

In [ ]:
crash_IVP = crash_risk(returns["IVP"].to_frame( ))
crash_IVP.round(4)

In [ ]:
plot_weights_returns(weights_IVP, returns["IVP"].to_frame( )) 

### 2.5 Random-Weighted Portfolios
<a id="subsection-25"></a>

In [ ]:
num_portfs = 10_000  
rand_portfs_weights = np.random.random(num_portfs * len(tickers)).reshape(num_portfs, len(tickers))

sum_weights = rand_portfs_weights.sum(axis = 1, keepdims = True)
rand_portfs_weights = rand_portfs_weights / sum_weights

In [ ]:
rand_portfs_returns = returns[tickers].dot(rand_portfs_weights.transpose( ))
rand_portfs_summary = risk_return(rand_portfs_returns)# .sort_values(by = "Annual Sharpe", ascending = False)

In [ ]:
rand_portfs_returns.round(4)

In [ ]:
rand_portfs_summary.round(4) 

### 2.6 Global Minimum (Annualized) Variance Portfolio 
<a id="subsection-26"></a>

In [ ]:
weights_MVP = rand_portfs_weights[rand_portfs_summary[("Annual",  "Volatility")].pow(2).idxmin( )]
weights_MVP = pd.DataFrame(np.tile(weights_MVP, (len(returns[tickers]), 1)), columns = tickers, index = returns.index)
returns["MVP"] = rand_portfs_returns[rand_portfs_summary[("Annual",  "Volatility")].pow(2).idxmin( )]

In [ ]:
summary_MVP = risk_return(returns["MVP"].to_frame( ))
summary_MVP.round(4)

In [ ]:
best_worst_MVP = best_worst_returns(returns["MVP"].to_frame( ))
best_worst_MVP.round(4) 

In [ ]:
win_percent_MVP = win_percentage(returns["MVP"].to_frame( ))  
win_percent_MVP.round(2)

In [ ]:
win_loss_MVP = win_loss_ratios(returns["MVP"].to_frame( )) 
win_loss_MVP.round(4)

In [ ]:
win_lose_streak_MVP = streak(returns["MVP"].to_frame( ))
win_lose_streak_MVP 

In [ ]:
dist_attributes_MVP = distribution_attributes(returns["MVP"].to_frame( ))
dist_attributes_MVP.round(4)

In [ ]:
downside_MVP = downside(returns["MVP"].to_frame( ))  
downside_MVP.round(4) 

In [ ]:
tail_MVP = tail_attributes(returns["MVP"].to_frame( ), threshold = 0.05)
tail_MVP.round(4) 

In [ ]:
VaR_MVP = Value_at_Risk(returns["MVP"].to_frame( ))
VaR_MVP.round(4)

In [ ]:
crash_MVP = crash_risk(returns["MVP"].to_frame( ))
crash_MVP.round(4)

In [ ]:
plot_weights_returns(weights_MVP, returns["MVP"].to_frame( ))

### 2.7 Maximum (Annualized) Sharpe Portfolio 
<a id="subsection-27"></a>

In [ ]:
weights_MSP = rand_portfs_weights[rand_portfs_summary[("Annual", "Sharpe")].idxmax( )]
weights_MSP = pd.DataFrame(np.tile(weights_MSP, (len(returns[tickers]), 1)), columns = tickers, index = returns.index)
returns["MSP"] = rand_portfs_returns[rand_portfs_summary[("Annual", "Sharpe")].idxmax( )]  

In [ ]:
summary_MSP = risk_return(returns["MSP"].to_frame( )) 
summary_MSP.round(4) 

In [ ]:
best_worst_MSP = best_worst_returns(returns["MSP"].to_frame( )) 
best_worst_MSP.round(4) 

In [ ]:
win_percent_MSP = win_percentage(returns["MSP"].to_frame( ))  
win_percent_MSP.round(2)

In [ ]:
win_loss_MSP = win_loss_ratios(returns["MSP"].to_frame( )) 
win_loss_MSP.round(4)

In [ ]:
win_lose_streak_MSP = streak(returns["MSP"].to_frame( ))
win_lose_streak_MSP 

In [ ]:
dist_attributes_MSP = distribution_attributes(returns["MSP"].to_frame( ))
dist_attributes_MSP.round(4)

In [ ]:
downside_MSP = downside(returns["MSP"].to_frame( ))  
downside_MSP.round(4) 

In [ ]:
tail_MSP = tail_attributes(returns["MSP"].to_frame( ), threshold = 0.05)
tail_MSP.round(4) 

In [ ]:
VaR_MSP = Value_at_Risk(returns["MSP"].to_frame( ))
VaR_MSP.round(4) 

In [ ]:
crash_MSP = crash_risk(returns["MSP"].to_frame( ))
crash_MSP.round(4)

In [ ]:
plot_weights_returns(weights_MSP, returns["MSP"].to_frame( ))

### 2.8 Variance Minimization using Scipy
<a id="subsection-28"></a>

In [ ]:
ann_returns = returns[tickers].mean( ) * 252 
cov_mat = returns[tickers].cov( )  * 252 

In [ ]:
def compute_variance(weights) :
    portf_var = np.dot(weights.T, np.dot(cov_mat, weights)) 
    return portf_var 

In [ ]:
def optimize_metric(cov_mat, func) :
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 1) for asset in range(len(cov_mat)))
    initial_weights = np.ones(len(cov_mat)) / len(cov_mat)
    
    result = optimize.minimize(func, initial_weights, method = "SLSQP", bounds = bounds, constraints = constraints)
    optimized_weights = result.x
    return optimized_weights 

In [ ]:
sp_weights_MVP = optimize_metric(cov_mat, compute_variance) 
sp_weights_MVP = pd.DataFrame(np.tile(sp_weights_MVP, (len(returns[tickers]), 1)), columns = tickers, index = returns.index)

sp_returns_MVP = returns[tickers].mul(sp_weights_MVP).sum(axis = 1)
sp_returns_MVP = pd.DataFrame(data = sp_returns_MVP, columns = ["scipy_MVP"])

In [ ]:
sp_weights_MVP.round(4)

In [ ]:
summary_sp_MVP = risk_return(sp_returns_MVP["scipy_MVP"].to_frame( )) 
MVP_comparison = pd.concat(axis = 0, objs = [summary_MVP, summary_sp_MVP])
MVP_comparison.round(4)

In [ ]:
plot_weights_returns(sp_weights_MVP, sp_returns_MVP["scipy_MVP"].to_frame( ))

### 2.9 Sharpe Maximization using Scipy
<a id="subsection-29"></a>

In [ ]:
def compute_sharpe(weights):
    portf_return = np.sum(weights * ann_returns)
    portf_std = np.sqrt(np.dot(weights.T, np.dot(cov_mat, weights)))
    sharpe = (portf_return - rf_rate) / portf_std
    return - sharpe 

In [ ]:
sp_weights_MSP = optimize_metric(cov_mat, compute_sharpe) 
sp_weights_MSP = pd.DataFrame(np.tile(sp_weights_MSP, (len(returns[tickers]), 1)), columns = tickers, index = returns.index)

sp_returns_MSP = returns[tickers].mul(sp_weights_MSP).sum(axis = 1)
sp_returns_MSP = pd.DataFrame(data = sp_returns_MSP, columns = ["scipy_MSP"])

In [ ]:
summary_sp_MSP = risk_return(sp_returns_MSP["scipy_MSP"].to_frame( )) 
MSP_comparison = pd.concat(axis = 0, objs = [summary_MSP, summary_sp_MSP])
MSP_comparison.round(4)

In [ ]:
plot_weights_returns(sp_weights_MSP, sp_returns_MSP["scipy_MSP"].to_frame( ))

### 2.10 Portfolio Comparison : Performance against Benchmark 
<a id="subsection-210"></a>

In [ ]:
portfs = ["PWP", "EWP", "MVWP", "IVP", "MVP", "MSP"]
cum_returns_portfs = (1 + returns[portfs]).cumprod( ) - 1 

In [ ]:
plot_cum_returns(cum_returns_portfs)

In [ ]:
plot_histogram(returns[portfs])

In [ ]:
plot_box_whisker(returns[portfs])  

In [ ]:
plot_box_whisker(returns[portfs], "weekly")  

In [ ]:
plot_box_whisker(returns[portfs], "monthly")  

In [ ]:
plot_box_whisker(returns[portfs], "quarterly")  

In [ ]:
plot_box_whisker(returns[portfs], "annual")  

In [ ]:
plot_dollar_triangle(returns[portfs])

In [ ]:
summary_portfs = pd.concat(objs = [summary_PWP, summary_EWP, summary_MVWP, summary_IVP, summary_MVP, summary_MSP], axis = 0)

best_worst_portfs = pd.concat(objs = [best_worst_PWP, best_worst_EWP, best_worst_MVWP, best_worst_IVP, best_worst_MVP, best_worst_MSP], axis = 0)

win_percent_portfs = pd.concat(objs = [win_percent_PWP, win_percent_EWP, win_percent_MVWP, win_percent_IVP, win_percent_MVP, win_percent_MSP], axis = 0)

win_loss_portfs = pd.concat(objs = [win_loss_PWP, win_loss_EWP, win_loss_MVWP, win_loss_IVP, win_loss_MVP, win_loss_MSP], axis = 0)

win_lose_streak_portfs = pd.concat(objs = [win_lose_streak_PWP, win_lose_streak_EWP, win_lose_streak_MVWP, win_lose_streak_IVP, win_lose_streak_MVP, win_lose_streak_MSP], axis = 0)

dist_attributes_portfs = pd.concat(objs = [dist_attributes_PWP, dist_attributes_EWP, dist_attributes_MVWP, dist_attributes_IVP, dist_attributes_MVP, dist_attributes_MSP], axis = 0)

downside_portfs = pd.concat(objs = [downside_PWP, downside_EWP, downside_MVWP, downside_IVP, downside_MVP, downside_MSP], axis = 0) 

tail_portfs = pd.concat(objs = [tail_PWP, tail_EWP, tail_MVWP, tail_IVP, tail_MVP, tail_MSP], axis = 0) 

VaR_portfs = pd.concat(objs = [VaR_PWP, VaR_EWP, VaR_MVWP, VaR_IVP, VaR_MVP, VaR_MSP], axis = 0) 

crash_portfs = pd.concat(objs = [crash_PWP, crash_EWP, crash_MVWP, crash_IVP, crash_MVP, crash_MSP], axis = 0) 

In [ ]:
summary_portfs.style\
              .format("{:.4f}")\
              .highlight_max(color = "lightgreen")\
              .highlight_min(color = "lightcoral")

In [ ]:
best_worst_portfs.style\
                 .format("{:.4f}")\
                 .highlight_max(color = "lightgreen")\
                 .highlight_min(color = "lightcoral")

In [ ]:
win_percent_portfs.round(2)

In [ ]:
win_loss_portfs.style\
               .format("{:.4f}")\
               .highlight_max(color = "lightgreen")\
               .highlight_min(color = "lightcoral")

In [ ]:
win_lose_streak_portfs 

In [ ]:
dist_attributes_portfs.round(2)

In [ ]:
plot_qq(returns[portfs])

In [ ]:
downside_portfs.style\
               .format("{:.4f}")\
               .highlight_max(color = "lightgreen")\
               .highlight_min(color = "lightcoral") 

In [ ]:
plot_drawdowns(returns[portfs]) 

In [ ]:
tail_portfs.style\
           .format("{:.4f}")\
           .highlight_max(color = "lightgreen")\
           .highlight_min(color = "lightcoral")

In [ ]:
VaR_portfs.style\
          .format("{:.4f}")\
          .highlight_max(color = "lightgreen")\
          .highlight_min(color = "lightcoral")

In [ ]:
plot_Value_at_Risk(VaR_portfs) 

In [ ]:
monte_carlo_simulation(returns[portfs])

In [ ]:
plot_extreme_value_theory(returns[portfs])

In [ ]:
crash_portfs.style\
            .format("{:.4f}")\
            .highlight_max(color = "lightgreen")

$$
\text{Annualized Active Return}_{portfolio} = \text{Annualized } \bar{\text{Return}}_{portfolio} - \text{Annualized } \bar{\text{Return}}_{benchmark}
$$

$$
\text{Annualized Tracking Error}_{portfolio} = \sqrt{\frac{\sum_{t=1}^{T} (\text{Return}_{portfolio,t} - \text{Return}_{benchmark,t})^2}{T}} \times \sqrt{252}
$$

$$
\text{Annualized Information Ratio}_{portfolio} = \frac{\text{Annualized Active Return}_{portfolio}}{\text{Annualized Tracking Error}_{portfolio}}
$$

$$
\text{Annualized M}^2 \text{ Ratio}_{portfolio} = \text{Annualized Sharpe}_{portfolio} \times \text{Annualized } \sigma_{benchmark} + \text{Risk-Free Rate} 
$$

$$
\text{Upside Capture}_{portfolio} = \frac{\left( \prod_{t=1}^{T} 1 + \max(0, \text{ Return}_{portfolio,t}) \right) - 1}{\left( \prod_{t=1}^{T} 1 + \max(0, \text{ Return}_{benchmark,t}) \right) - 1} 
$$ 

$$
\text{Downside Capture}_{portfolio} = \frac{\left( \prod_{t=1}^{T} 1 + \min(0, \text{ Return}_{portfolio,t}) \right) - 1}{\left( \prod_{t=1}^{T} 1 + \min(0, \text{ Return}_{benchmark,t}) \right) - 1} 
$$ 

$$
\text{R}^2_{portfolio} = 1 - \frac{\sum_{t=1}^{T} (\text{Return}_{benchmark, t} - \hat{\text{Return}}_{benchmark, t})^2}{\sum_{t=1}^{T} (\text{Return}_{benchmark, t} - \bar{\text{Return}}_{benchmark, t})^2}
$$

In [ ]:
def perf_against_benchmark(returns_portfs_df, benchmark = "^NDXT") :
    """
    Arguments :
    - returns_portfs_df : DataFrame of portfolios' daily returns 
    - benchmark : str for the benchmark ticker (^NDXT by default)
    
    Returns :
    - perf_df : DataFrame displaying ratios of performance relative to a benchmark 
    
    Function :
    - Computes the annualized active return, annualized tracking error, annualized information ratio, annualized m-squared ratio, upside cature, downside capture, and r-squared 
    """
    compare = yf.download(tickers = benchmark, start = start_date, end = end_date)
    close_compare = compare["Close"].copy( ) 
    benchmark_returns = close_compare.pct_change(periods = 1).iloc[1:] 
    
    active_return_list = [  ]
    tracking_error_list = [  ]
    information_list = [  ]
    m2_list = [  ]
    upside_capture_list = [  ]
    downside_capture_list = [  ]
    r2_list = [  ]
    for portf in returns_portfs_df.columns :
        
        ann_return_portf = returns_portfs_df[portf].mean( ) * 252
        ann_return_benchmark = benchmark_returns.mean( ) * 252
        active_return = ann_return_portf - ann_return_benchmark
        
        returns_above_benchmark = returns_portfs_df[portf].sub(benchmark_returns)
        tracking_error = returns_above_benchmark.std( ) * np.sqrt(252)
        
        information = active_return / tracking_error 
        
        ann_sharpe_portf = (ann_return_portf - rf_rate) / (returns_portfs_df[portf].std( ) * np.sqrt(252)) 
        ann_std_benchmark = benchmark_returns.std( ) * np.sqrt(252)
        m2 = ann_sharpe_portf * ann_std_benchmark + rf_rate 
        
        cum_positive_portf = (1 + returns_portfs_df[portf].loc[returns_portfs_df[portf] > 0]).prod( ) - 1
        cum_positive_benchmark = (1 + benchmark_returns.loc[benchmark_returns > 0]).prod( ) - 1
        upside_capture = cum_positive_portf / cum_positive_benchmark
        
        cum_negative_portf = (1 + returns_portfs_df[portf].loc[returns_portfs_df[portf] < 0]).prod( ) - 1
        cum_negative_benchmark = (1 + benchmark_returns.loc[benchmark_returns < 0]).prod( ) - 1
        downside_capture = cum_negative_portf / cum_negative_benchmark
        
        lin_reg = linear_model.LinearRegression( )
        lin_reg.fit(returns_portfs_df[portf].to_frame( ) , benchmark_returns)
        Y_pred = lin_reg.predict(returns_portfs_df[portf].to_frame( ))
        r2 = metrics.r2_score(benchmark_returns, Y_pred)
        
        active_return_list.append(active_return)
        tracking_error_list.append(tracking_error)
        information_list.append(information)
        m2_list.append(m2)
        upside_capture_list.append(upside_capture)
        downside_capture_list.append(downside_capture)
        r2_list.append(r2)
    
    perf_df = pd.DataFrame({"Annual Active Return" : active_return_list, "Annual Tracking Error" : tracking_error_list, 
                            "Annual Information Ratio" : information_list, "Annual M2 Ratio" : m2_list,
                            "Upside Capture" : upside_capture_list, "Downside Capture" : downside_capture_list,  
                            "R-Squared" : r2_list}, 
                           index = returns_portfs_df.columns)
    print(f"Benchmark = {benchmark}")
    return perf_df 

In [ ]:
relative_perf_porfs = perf_against_benchmark(returns[portfs])

In [ ]:
relative_perf_porfs.style\
                   .format("{:.4f}")\
                   .highlight_max(color = "lightgreen")\
                   .highlight_min(color = "lightcoral")

### 2.11 Plotting the Efficient Frontier and the Capital Asset Line 
<a id="subsection-211"></a>

In [ ]:
plt.figure(figsize = (12 , 7))  
    
# plot the randomly–weighted portfolios 
plt.scatter(x = rand_portfs_summary[("Annual", "Volatility")], y = rand_portfs_summary[("Annual", "Return")], 
            c = rand_portfs_summary[("Annual", "Sharpe")], cmap = "RdYlGn", vmin = 0.5, vmax = 1.4, s = 20)

# plot the minimum variance portfolio 
plt.scatter(x = summary_portfs.loc["MVP" , ("Annual", "Volatility")], y = summary_portfs.loc["MVP" , ("Annual", "Return")], s = 30, c = "blue") 

plt.annotate("MVP", size = 8, c = "blue", 
             xy = (summary_portfs.loc["MVP" , ("Annual", "Volatility")] - 0.03, summary_portfs.loc["MVP" , ("Annual", "Return")]))

# plot the maximum sharpe portfolio 
plt.scatter(x = summary_portfs.loc["MSP" , ("Annual", "Volatility")], y = summary_portfs.loc["MSP" , ("Annual", "Return")],  
            c = "fuchsia", s = 100, marker = "*")

plt.annotate("MSP", size = 10, c = "fuchsia", 
             xy = (summary_portfs.loc["MSP" , ("Annual", "Volatility")] - 0.01, summary_portfs.loc["MSP" , ("Annual", "Return")] + 0.02))

# plot the price–weighted portfolio
plt.scatter(x = summary_portfs.loc["PWP" , ("Annual", "Volatility")], y = summary_portfs.loc["PWP" , ("Annual", "Return")], s = 30, c = "blue")

plt.annotate("PWP", size = 8, c = "blue", 
             xy = (summary_portfs.loc["PWP" , ("Annual", "Volatility")] + 0.01, summary_portfs.loc["PWP" , ("Annual", "Return")]))

# plot the equally–weighted portfolio
plt.scatter(x = summary_portfs.loc["EWP" , ("Annual", "Volatility")], y = summary_portfs.loc["EWP" , ("Annual", "Return")], s = 30, c = "blue")

plt.annotate("EWP", size = 8, c = "blue", 
             xy = (summary_portfs.loc["EWP" , ("Annual", "Volatility")] - 0.01, summary_portfs.loc["EWP" , ("Annual", "Return")] + 0.015))

# plot the market–value–weighted portfolio 
plt.scatter(x = summary_portfs.loc["MVWP" , ("Annual", "Volatility")], y = summary_portfs.loc["MVWP" , ("Annual", "Return")], c = "blue" , s = 30)

plt.annotate("MVWP", size = 8, c = "blue", 
             xy = (summary_portfs.loc["MVWP", ("Annual", "Volatility")] - 0.04, summary_portfs.loc["MVWP" , ("Annual", "Return")]))  

# plot the inverse volatility portfolio 
plt.scatter(x = summary_portfs.loc["IVP" , ("Annual", "Volatility")], y = summary_portfs.loc["IVP" , ("Annual", "Return")], c = "blue" , s = 30)

plt.annotate("IVP", size = 8, c = "blue", 
             xy = (summary_portfs.loc["IVP", ("Annual", "Volatility")] - 0.02, summary_portfs.loc["IVP" , ("Annual", "Return")] + 0.015))  

# plot the individual stocks in the portfolio 
plt.scatter(x = summary_stocks[("Annual", "Volatility")], y = summary_stocks[("Annual", "Return")], s = 30, c = "black")

for ticker in tickers : 
    plt.annotate(ticker, xy = (summary_stocks.loc[ticker , ("Annual", "Volatility")] - 0.01, summary_stocks.loc[ticker , ("Annual", "Return")] + 0.015), size = 8)  

# plot the risk free asset and the capital market line 
plt.scatter(x = pd.Series(data = 0).values, y = rf_rate, s = 30, c = "fuchsia")
plt.annotate("risk-free asset", size = 10, c = "fuchsia", xy = (rf_rate, rf_rate))

plt.plot([0, summary_portfs.loc["MSP" , ("Annual", "Volatility")]], [rf_rate, summary_portfs.loc["MSP" , ("Annual", "Return")]], c = "fuchsia", marker = "o", linestyle = "-")
plt.annotate("CAL", size = 10, c = "fuchsia", xy = (0.1, 0.2))

plt.xticks(ticks = np.array(range(0, 71, 5)) / 100)
plt.xlabel(xlabel = "Annualized Volatility", fontsize = 12)
plt.ylabel(ylabel = "Annualized Return", fontsize = 12) 
plt.grid( )
plt.show( )   

## 3. Putting It All Together 
<a id="section-3"></a>

In [ ]:
summary = pd.concat(objs = [summary_stocks, summary_portfs], axis = 0) 

best_worst = pd.concat(objs = [best_worst_stocks, best_worst_portfs], axis = 0) 

win_percent = pd.concat(objs = [win_percent_stocks, win_percent_portfs], axis = 0)

win_loss = pd.concat(objs = [win_loss_stocks, win_loss_portfs], axis = 0)

win_lose_streak = pd.concat(objs = [win_lose_streak_stocks, win_lose_streak_portfs], axis = 0)

dist_attributes = pd.concat(objs = [dist_attributes_stocks, dist_attributes_portfs], axis = 0)

downside = pd.concat(objs = [downside_stocks, downside_portfs], axis = 0) 

tail = pd.concat(objs = [tail_stocks, tail_portfs], axis = 0) 

VaR = pd.concat(objs = [VaR_stocks, VaR_portfs], axis = 0) 

crash = pd.concat(objs = [crash_stocks, crash_portfs], axis = 0) 

In [ ]:
summary.style\
       .format("{:.4f}")\
       .highlight_max(color = "lightgreen")\
       .highlight_min(color = "lightcoral")

In [ ]:
best_worst.style\
          .format("{:.4f}")\
          .highlight_max(color = "lightgreen")\
          .highlight_min(color = "lightcoral")

In [ ]:
win_percent.round(2)

In [ ]:
win_loss.style\
        .format("{:.4f}")\
        .highlight_max(color = "lightgreen")\
        .highlight_min(color = "lightcoral")

In [ ]:
win_lose_streak

In [ ]:
dist_attributes.round(2)

In [ ]:
downside.style\
        .format("{:.4f}")\
        .highlight_max(color = "lightgreen")\
        .highlight_min(color = "lightcoral")

In [ ]:
tail.style\
    .format("{:.4f}")\
    .highlight_max(color = "lightgreen")\
    .highlight_min(color = "lightcoral") 

In [ ]:
VaR.style\
   .format("{:.4f}")\
   .highlight_max(color = "lightgreen")\
   .highlight_min(color = "lightcoral") 

In [ ]:
crash.style\
     .format("{:.4f}")\
     .highlight_max(color = "lightgreen")

### Impact and Magnitude of Diversification 
<a id="subsection-31"></a>

$$
\text{Diversification Ratio}_{t} = \frac{\sigma_{portfolio,t}}{\sum_{i=1}^{N} \sigma_{i,t} \times \text{Weight}_{i,t}} - 1
$$

$$
\text{Herfindahl-Hirschman Index}_{t} = {\sum_{i=1}^{N} (\text{Weight}_{i,t})^2} 
$$

In [ ]:
def plot_portf_diversification(returns_tickers, weights_portf, returns_portf) :
    """
    Arguments :
    - returns_tickers : DataFrame of daily returns for assets in the portfolio 
    - weights_portf : DataFrame of monthly weights for assets in the portfolio
    - returns_portf : DataFrame of daily returns for the portfolio 
    
    Returns :
    - None 
        
    Function :
    - Draws a line plot of the monthly weighted sum of assets' volatilities 
    - Draws a line plot of the monthly volatility of the portfolio
    - Draws a line plot of the % difference between portfolio's volatility and assets' volatilities 
    - Draws a line plot of the sum of squared monthly weights 
    """
    monthly_std_tickers = returns_tickers.resample("BM").std( )
    monthly_weights = weights_portf.resample("BM").last( )
    weighted_sum_std = monthly_std_tickers.mul(monthly_weights).sum(axis = 1)
    
    portf = returns_portf.name 
    monthly_std_portf = returns_portf.resample("BM").std( )
    
    diversif_ratio = - (monthly_std_portf.sub(weighted_sum_std)).div(weighted_sum_std)
    herf_hirsch_index = monthly_weights.pow(2).sum(axis = 1)
    
    fig , ax = plt.subplots(figsize = (11.5 , 14), sharex = False, sharey = False, nrows = 2, ncols = 1)
    
    ax[0].plot(weighted_sum_std.mul(100), label = "Weighted Sum of Assets's Volatilities", c = "red")
    ax[0].plot(monthly_std_portf.mul(100), label = f"{portf} Volatility", c = "orange")
    ax[0].set_ylabel("Monthly (%)", fontsize = 11)
    ax[0].legend( )
    
    ax[1].plot(diversif_ratio.mul(100), linestyle = "dotted", marker = "o", color = "tab:green", label = "Diversification Ratio")
    ax[1].set_ylabel("Monthly (%)", fontsize = 11)
    
    ax2 = ax[1].twinx( )
    ax2.plot(herf_hirsch_index.mul(100), linestyle = "dashed", marker = "s", color = "tab:blue", label = "Herfindahl-Hirschman Index")
    ax2.set_ylabel("Monthly (%)", fontsize = 11)
    
    lines1, labels1 = ax[1].get_legend_handles_labels( )
    lines2, labels2 = ax2.get_legend_handles_labels( )
    ax[1].legend(lines1 + lines2, labels1 + labels2, loc = "best")
    
    plt.show( )

In [ ]:
plot_portf_diversification(returns[tickers], weights_PWP, returns["PWP"])

In [ ]:
plot_portf_diversification(returns[tickers], weights_EWP, returns["EWP"])

In [ ]:
plot_portf_diversification(returns[tickers], weights_MVWP, returns["MVWP"])

In [ ]:
plot_portf_diversification(returns[tickers], weights_IVP, returns["IVP"])

In [ ]:
plot_portf_diversification(returns[tickers], weights_MVP, returns["MVP"])

In [ ]:
plot_portf_diversification(returns[tickers], weights_MSP, returns["MSP"])

## 4. Capital Asset Pricing Model 
<a id="section-4"></a>

### 4.1 Market Portfolio and Annualized Covariance
<a id="subsection-41"></a>

$$
\text{Annualized Cov}(\text{Return}_X, \text{Return}_Y) = \left( \frac{1}{T-1} \sum_{t=1}^{T} (\text{Return}_{X,t} - \bar{\text{Return}}_{X}) \times (\text{Return}_{Y,t} - \bar{\text{Return}}_{Y}) \right) \times 252 
$$

In [ ]:
sp500 = yf.download(tickers = "^GSPC", start = start_date, end = end_date)
returns["MP"] = sp500["Close"].pct_change(periods = 1).dropna( ) 
cov_table = returns.cov( ) * 252 

summary_MP = risk_return(returns["MP"].to_frame( ))
summary = pd.concat(objs = [summary, summary_MP], axis = 0) 

In [ ]:
ann_summary = summary["Annual"].copy( )
ann_summary.columns = ["Annual Return", "Annual Volatility", "Annual Sharpe"]

In [ ]:
cov_table.round(4)

### 4.2 Beta, Systematic Risk, and Idiosyncratic Risk 
<a id="subsection-42"></a>

$$
\left( \text{Total Risk}_S \right)^2 = \left( \text{Annualized } \sigma_S \right)^2
$$ 

$$
\left( \text{Systematic Risk}_S \right)^2 = \text{Cov}(\text{Return}_{S}, \text{Return}_M) \times 252
$$ 

$$
\left( \text{Unsystematic Risk}_S \right)^2 = \left( \text{Total Risk}_S \right)^2 - \left( \text{Systematic Risk}_S \right)^2
$$ 

$$
\text{Annualized } \beta_S = \frac{\text{Annualized Cov}(\text{Return}_{S}, \text{Return}_M)}{\text{Annualized } \sigma^2_M}
$$

In [ ]:
ann_summary["Systematic Risk"] = cov_table["MP"].apply(np.sqrt) 
ann_summary["Unsystematic Risk"] = (ann_summary["Annual Volatility"] ** 2 - ann_summary["Systematic Risk"] ** 2).apply(np.sqrt) 
ann_summary["Beta"] = cov_table["MP"].div(cov_table.loc["MP" , "MP"]) 

In [ ]:
ann_summary 

In [ ]:
ann_summary[["Beta", "Systematic Risk", "Unsystematic Risk"]].style\
                                                             .format("{:.4f}")\
                                                             .highlight_max(color = "lightgreen")\
                                                             .highlight_min(color = "lightcoral")  

In [ ]:
ann_summary[["Systematic Risk" , "Unsystematic Risk"]].mul(100).plot(kind = "bar", figsize = (12 , 8), fontsize = 10, 
                                                                     rot = 90, yticks = range(0, 66, 5), color = ["orange", "red"])  

plt.ylabel(ylabel = "Annualized Risk (%)", fontsize = 12) 
plt.style.use("default")   
plt.show( ) 

### 4.3 Rolling Beta 
<a id="subsection-43"></a>

In [ ]:
def plot_rolling_beta(returns_df, window = 3) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    - window : int for the interval of the rolling window (months)
    
    Returns :
    - None 
    
    Function :
    - Plots the rolling beta for each asset (3-month by default)
    """
    sp500 = yf.download(tickers = "^GSPC", start = start_date, end = end_date)
    sp500_returns = sp500["Close"].pct_change(periods = 1).dropna( )
    sp500_monthly_returns = sp500_returns.resample("BM").agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1)
    
    monthly_returns = returns_df.resample("BM").agg(lambda daily_ret: (1 + daily_ret).prod( ) - 1)
    
    rolling_cov = monthly_returns.rolling(window).cov(sp500_monthly_returns) 
    rolling_var = sp500_monthly_returns.rolling(window).var( )
    rolling_beta = rolling_cov.div(rolling_var, axis = 0) 
    
    assets = returns_df.columns 
    n_assets = len(returns_df.columns)
    fig, ax = plt.subplots(n_assets, 1, figsize = (12, n_assets * 4)) 
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
    
    for i in range(n_assets) :
        ax[i].plot(rolling_beta[assets[i]], color = colors[i], label = assets[i])
        ax[i].set_ylabel(ylabel = "Rolling Beta", fontsize = 12)
        ax[i].legend( )
    
    plt.style.use("default") 
    plt.show( )

In [ ]:
plot_rolling_beta(returns[tickers])

In [ ]:
plot_rolling_beta(returns[portfs])

### 4.4 Expected Return and Alpha 
<a id="subsection-44"></a>

$$
\text{Expected Return} = \text{Risk-Free Rate} + \beta \times (\text{Return}_M - \text{Risk-Free Rate})
$$ 

$$
\text{Alpha} = \text{Realized Return} - \text{Expected Return}
$$ 

In [ ]:
ann_summary["CAPM Return"] = rf_rate + (ann_summary.loc["MP" , "Annual Return"] - rf_rate) * ann_summary["Beta"] 
ann_summary["Alpha"] = ann_summary["Annual Return"].sub(ann_summary["CAPM Return"])

In [ ]:
ann_summary[["Annual Return", "CAPM Return", "Alpha"]].style\
                                                      .format("{:.4f}")\
                                                      .highlight_max(color = "lightgreen")\
                                                      .highlight_min(color = "lightcoral")

### 4.5 Security Market Line 
<a id="subsection-45"></a>

In [ ]:
plt.figure(figsize = (12 , 8))

# plot the price–weighted portfolio
plt.scatter(x = ann_summary.loc["PWP" , "Beta"], y = ann_summary.loc["PWP" , "Annual Return"], s = 30, c = "blue")

plt.annotate("PWP", size = 8, c = "blue", 
             xy = (ann_summary.loc["PWP" , "Beta"] + 0.02, ann_summary.loc["PWP" , "Annual Return"]))

# plot the equally–weighted portfolio
plt.scatter(x = ann_summary.loc["EWP" , "Beta"], y = ann_summary.loc["EWP" , "Annual Return"], s = 30, c = "blue")

plt.annotate("EWP", size = 8, c = "blue", 
             xy = (ann_summary.loc["EWP" , "Beta"] - 0.035, ann_summary.loc["EWP" , "Annual Return"] + 0.01))

# plot the market-value–weighted portfolio
plt.scatter(x = ann_summary.loc["MVWP" , "Beta"], y = ann_summary.loc["MVWP" , "Annual Return"], s = 30, c = "blue")

plt.annotate("MVWP", size = 8, c = "blue", 
             xy = (ann_summary.loc["MVWP" , "Beta"] + 0.02, ann_summary.loc["MVWP" , "Annual Return"]))

# plot the market portfolio
plt.scatter(x = ann_summary.loc["MP", "Beta"], y = ann_summary.loc["MP", "Annual Return"], s = 30, c = "green")

plt.annotate("MP", size = 10, c = "green", 
             xy = (ann_summary.loc["MP" , "Beta"] - 0.015, ann_summary.loc["MP" , "Annual Return"] + 0.01))

# plot the minimum variance portfolio
plt.scatter(x = ann_summary.loc["MVP", "Beta"], y = ann_summary.loc["MVP", "Annual Return"], s = 30, c = "blue")

plt.annotate("MVP", size = 8, c = "blue", 
             xy = (ann_summary.loc["MVP" , "Beta"] - 0.02, ann_summary.loc["MVP" , "Annual Return"] - 0.025))

# plot the maximum sharpe portfolio 
plt.scatter(x = ann_summary.loc["MSP", "Beta"], y = ann_summary.loc["MSP", "CAPM Return"], c = "fuchsia", s = 30)

plt.annotate("MSP", size = 8, c = "fuchsia", 
             xy = (ann_summary.loc["MSP" , "Beta"] - 0.025, ann_summary.loc["MSP" , "CAPM Return"] + 0.01))

# plot the inverse volatility portfolio 
plt.scatter(x = ann_summary.loc["IVP", "Beta"], y = ann_summary.loc["IVP", "CAPM Return"], c = "blue", s = 30)

plt.annotate("IVP", size = 8, c = "blue", 
             xy = (ann_summary.loc["IVP" , "Beta"] - 0.025, ann_summary.loc["IVP" , "CAPM Return"] + 0.01))

# plot the individual stocks in the portfolio 
plt.scatter(x = ann_summary.loc[tickers , "Beta"], y = ann_summary.loc[tickers, "Annual Return"], s = 30, c = "black")

for ticker in tickers : 
    plt.annotate(ticker, xy = (ann_summary.loc[ticker , "Beta"] - 0.03, ann_summary.loc[ticker , "Annual Return"] + 0.01), size = 8)  

#  plot the risk–free asset and the security market line 
plt.scatter(x = pd.Series(data = 0).values, y = rf_rate, s = 30, c = "green")
plt.annotate("risk-free asset", size = 10, c = "green", xy = (pd.Series(data = 0).values - 0.04, rf_rate + 0.03))

betas = np.array(range(0, 180, 10)) / 100 
capm_returns = rf_rate + (ann_summary.loc["MP" , "Annual Return"] - rf_rate) * betas
plt.plot(betas, capm_returns, c = "green")

plt.fill_between(betas, capm_returns, color = "red", alpha = 0.3)
plt.fill_between(betas, 0.8, color = "green", alpha = 0.2)
plt.annotate("SML", size = 10, c = "green", xy = (0.4, 0.10))

plt.xticks(ticks = np.array(range(0, 18)) / 10)
plt.yticks(ticks = np.array(range(0, 81, 5)) / 100)
plt.xlabel(xlabel = "Annualized Beta",  fontsize = 12)
plt.ylabel(ylabel = "Annualized Return",  fontsize = 12) 
plt.grid( )
plt.style.use("default")  
plt.show( )

## 5. Farma-French 5-Factor Model 
<a id="section-5"></a>

### 5.1 Factor Data 
<a id="subsection-51"></a>

In [ ]:
ff_five_factor = web.DataReader("F-F_Research_Data_5_Factors_2x3_Daily", "famafrench", start = start_date, end = end_date)[0]
ff_five_factor = ff_five_factor.iloc[1: , :-1].copy( )
ff_five_factor.columns = ["Excess Market Return", "Small minus Big", "High minus Low", "Robust minus Weak", "Conservative Minus Aggressive"]

In [ ]:
ff_five_factor.describe( ).round(4)

### 5.2 Rolling Correlation With Factors
<a id="subsection-52"></a>

In [ ]:
def plot_rolling_corr(returns_df, ff_factor_df, window = 21) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns of the asset 
    - ff_factor_df : DataFrame of daily factor data 
    - window : int for the interval of the rolling window (days)
    
    Returns :
    - None 
    
    Function :
    - Plots the rolling correlation between the asset and each factor (21 days by default)
    """
    asset = returns_df.name
    rolling_corr = returns_df.rolling(window).corr(ff_factor_df) 
    
    rolling_corr.plot(figsize = (12, 15), fontsize = 10, subplots = True, sharex = False, xlabel = "", 
                      title = [f"Rolling Correlation Between {asset} and FF Factors", "", "", "", ""])
    
    plt.style.use("default")  
    plt.show( )

In [ ]:
plot_rolling_corr(returns["AAPL"], ff_five_factor)

In [ ]:
plot_rolling_corr(returns["NVDA"], ff_five_factor)

In [ ]:
plot_rolling_corr(returns["MVP"], ff_five_factor)

In [ ]:
plot_rolling_corr(returns["MSP"], ff_five_factor)

### 5.3 Linear Regression 
<a id="subsection-53"></a>

$$
\hat{R}_t = \beta_{0} + \beta_{1} \times (\text{MKT}_t - \text{RF}_t) + \beta_{2} \times \text{SMB}_t + \beta_{3} \times \text{HML}_t + \beta_{4} \times \text{RMW}_t + \beta_{5} \times \text{CMA}_t + \epsilon_{t}
$$

In [ ]:
def farma_french(returns_df, ff_factor_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    - ff_factor_df : DataFrame of daily factor data 
    
    Returns :
    - ff_returns_df : DataFrame of daily returns predicted by the French-Farma model 
    - ff_coefs_df : DataFrame of regression coefficients 
    
    Function :
    - Fits a linear regression model with factor data as predictor variables and returns as the target variable  
    - Retrieves the regression coefficients of each factor for each asset 
    """
    X = ff_factor_df.copy( )
    Y = returns_df.copy( )
    
    lin_reg = linear_model.LinearRegression(fit_intercept = True)
    lin_reg.fit(X , Y)
    Y_pred = lin_reg.predict(X) 
    
    coefs = lin_reg.coef_
    coefs_columns = [f"Coef_{factor}" for factor in ff_factor_df.columns] 
    
    ff_returns_df = pd.DataFrame(Y_pred, index = returns_df.index, columns = returns_df.columns)
    ff_coefs_df = pd.DataFrame(coefs, index = returns_df.columns, columns = coefs_columns)
    return ff_returns_df, ff_coefs_df  

In [ ]:
ff_returns = farma_french(returns, ff_five_factor)[0]

In [ ]:
ff_returns.round(4)

In [ ]:
def regression_summary(returns_df, ff_factor_df) :
    """
    Arguments :
    - returns_df : DataFrame of daily returns 
    - ff_factor_df : DataFrame of daily factor data 
    
    Returns :
    - None  
    
    Function :
    - Prints the ANOVA table and summary statistics of the fitted French-Farma regression model for each asset 
    """
    X = ff_factor_df.to_numpy( )
    X = tools.add_constant(X)
    
    for asset in returns_df.columns :
        Y = returns_df[asset].to_numpy( )
        lin_reg = regression.linear_model.OLS(endog = Y, exog = X).fit( )
        summary = lin_reg.summary( )
        summary.tables[0].title = asset
        print(summary)
        print("")

In [ ]:
regression_summary(returns[tickers], ff_five_factor)

In [ ]:
regression_summary(returns[portfs + ["MP"]], ff_five_factor)

### 5.4 Regression Coefficients : Magnitude of Factors Driving Returns
<a id="subsection-54"></a>

In [ ]:
ff_coefs = farma_french(returns, ff_five_factor)[1]

In [ ]:
ff_coefs.style\
        .format("{:.4f}")\
        .highlight_max(color = "lightgreen")\
        .highlight_min(color = "lightcoral")

In [ ]:
def plot_ff_coefs(ff_coefs_df) :
    """
    Arguments :
    - ff_coefs_df : DataFrame of regression coefficients of each factor for each asset 
    
    Returns :
    - None 
    
    Function :
    - Plots bar charts of the regression coefficients of each factor for each asset 
    """
    ff_coefs_df.mul(100).plot(kind = "bar", figsize = (12 , 25), fontsize = 10, width = 0.4, rot = 0, subplots = True, 
                              sharex = False, xlabel = "", ylabel = "% Change in Return", legend = False) 

    plt.style.use("default")
    plt.show( )

In [ ]:
plot_ff_coefs(ff_coefs.loc[tickers])

In [ ]:
plot_ff_coefs(ff_coefs.loc[portfs + ["MP"]])

### 5.5 Model Evaluation and Residual Analysis
<a id="subsection-55"></a>

$$
\text{Mean Squared Error} = \frac{1}{T} \sum_{t=1}^{T} (\text{Return}_t - \hat{\text{Return}}_t)^2
$$

$$
\text{Mean Absolute Error} = \frac{1}{T} \sum_{t=1}^{T} |\text{Return}_t - \hat{\text{Return}}_t|
$$

$$ 
\text{Mean Absolute Percentage Error} = \frac{1}{T} \sum_{t=1}^{T} \left| \frac{\text{Return}_t - \hat{\text{Return}}_t}{\text{Return}_t} \right| \times 100
$$

$$
\text{R}^2 = 1 - \frac{{\sum_{t=1}^{T} (\text{Return}_t - \hat{\text{Return}}_t)^2}}{{\sum_{t=1}^{T} (\text{Return}_t - \bar{\text{Return}})^2}}
$$

$$
\text{Adjusted R}^2 = 1 - \frac{{(1 - R^2) \times (T - 1)}}{{T - 5 - 1}}
$$

$$
\text{Akaike Information Criterion} = T \times \log\left(\frac{SSE}{T}\right) + 2 \times 5
$$

$$
\text{Bayesian Information Criterion} = T \times \log\left(\frac{SSE}{T}\right) + 5 \times \log(T)
$$

In [ ]:
def eval_perf(Y, Y_pred) :
    """
    Arguments :
    - Y : DataFrame of actual daily returns 
    - Y_pred : DataFrame of daily returns predicted by the French-Farma model 
    
    Returns :
    - perf_df : DataFrame displaying ratios for evaluating a regression model 
    
    Function :
    - Computes the mean squared error, mean absolute error, mean absolute percentage error, r-squared, adjusted r-squared, akaike information criterion, and bayesian information criterion 
    """
    perf_metrics = {  }
    for asset in Y.columns :
        
        mse = metrics.mean_squared_error(Y[asset], Y_pred[asset])
        mae = metrics.mean_absolute_error(Y[asset], Y_pred[asset]) 
        mape = metrics.mean_absolute_percentage_error(Y[asset], Y_pred[asset]) 
        r2 = metrics.r2_score(Y[asset], Y_pred[asset])
        adjusted_r2 = 1 - ((1 - r2) * (len(Y) - 1) / (len(Y) - 5 - 1))
        
        residuals = Y[asset] - Y_pred[asset] 
        sse = np.sum(residuals ** 2)
        aic = len(Y[asset]) * np.log(sse / len(Y[asset])) + 2 * 5   
        bic = len(Y[asset]) * np.log(sse / len(Y[asset])) + 5 * np.log(len(Y[asset])) 
        
        perf_metrics[asset] = [mse, mae, mape, r2, adjusted_r2, aic, bic] 
    
    perf_metrics_df = pd.DataFrame(perf_metrics, index = ["Mean Squared Error", "Mean Absolute Error", "Mean Absolute Percentage Error", 
                                                          "R-Squared", "Adjusted R-Squared", 
                                                          "Akaike Information Criterion", "Bayesian Information Criterion"]).T    
    return perf_metrics_df 

In [ ]:
ff_model_perf = eval_perf(returns, ff_returns)

In [ ]:
ff_model_perf.round(4)

$$
e_t = y_t - \hat{y}_t 
$$ 

$\text{Where :}$
- $ y_t \text{ represents the actual return at time \( t \).} $
- $ \hat{y}_t \text{ represents the predicted return at time \( t \).} $ 

$$
DW = \frac{\sum_{t=2}^{T} (e_t - e_{t-1})^2}{\sum_{t=1}^{T} e_t^2} 
$$

$\text{Where :}$
- $ e_t \text{ represents the residual at time \( t \).} $
- $ T \text{ is the number of periods.} $ 

In [ ]:
def residuals_summary(Y, Y_pred) :
    """
    Arguments :
    - Y : DataFrame of actual daily returns 
    - Y_pred : DataFrame of daily returns predicted by French-Farma model 
    
    Returns :
    - None     
    
    Function :
     - Computes the difference between actual daily returns and predicted daily returns 
    - Prints the average, standard deviation, minimum, 25th percentile, median, 75th percentile, and maximum of residuals 
    """
    residuals = Y - Y_pred 
    summary_df = residuals.mul(100).describe( ).T.drop(columns = "count")
    summary_df.columns = ["Average", "Standard Deviation", "Minimum", "25th Percentile", "Median", "75th Percentile", "Maximum"]
    summary_df.columns = [f"{col} (%)" for col in summary_df.columns]
    
    print("Summary Statistics of French-Farma Regression Residuals (%)")
    return summary_df 

In [ ]:
residuals_summary(returns, ff_returns).round(4)

In [ ]:
def plot_residuals(Y, Y_pred) :
    """
    Arguments :
    - Y : DataFrame of actual daily returns 
    - Y_pred : DataFrame of daily returns predicted by French-Farma model 
    
    Returns :
    - None 
    
    Function :
    - Computes the difference between actual daily returns and predicted daily returns 
    - Plots line graphs of residuals over time for each asset 
    - Plots scatter points for the 10 largest positive residuals and 10 largest negative residuals 
    """
    residuals = Y - Y_pred 
    
    n_assets = len(Y.columns)
    fig , ax = plt.subplots(figsize = (12 , n_assets * 5), sharex = False, sharey = False, nrows = n_assets, ncols = 1)
    
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
    for i, asset in enumerate(Y.columns) :
        
        ax[i].plot(residuals[asset] * 100, color = colors[i], label = asset)
        dw_statistic = stattools.durbin_watson(residuals[asset])
        legend_label = f"{asset}\nDurbin-Watson Statistic = {dw_statistic:.2f}"
        
        largest_positive_resids = residuals[asset].nlargest(10).sort_index( ) 
        largest_negative_resids = residuals[asset].nsmallest(10).sort_index( ) 
        ax[i].scatter(largest_positive_resids.index, largest_positive_resids.values * 100, color = colors[i], s = 30)
        ax[i].scatter(largest_negative_resids.index, largest_negative_resids.values * 100, color = colors[i], s = 30)
        
        ax[i].set_ylabel(ylabel = "Residual (%)", fontsize = 12)
        ax[i].legend([legend_label], fontsize = 10)
    
    plt.style.use("default")
    plt.show( )

In [ ]:
plot_residuals(returns[tickers], ff_returns[tickers])

In [ ]:
plot_residuals(returns[portfs + ["MP"]], ff_returns[portfs + ["MP"]])

In [ ]:
def plot_fitted_vs_actual(Y, Y_pred, cumulative = False) :
    """
    Arguments :
    - Y : DataFrame of actual daily returns 
    - Y_pred : DataFrame of daily returns predicted by French-Farma model 
    - cumulative : boolean indicating whether to compute cumulative returns (False by default)
    
    Returns :
    - None 
    
    Function :
    - Plots the line graph of predicted daily returns and scatter points of actual daily returns for each asset 
    - Plots the ligne graphs of predicted cumulative returns and actual cumulative returns for each asset 
    """
    residuals = Y - Y_pred 
    
    n_assets = len(Y.columns)
    fig , ax = plt.subplots(figsize = (12 , n_assets * 5), sharex = False, sharey = False, nrows = n_assets, ncols = 1)
    
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
    for i, asset in enumerate(Y.columns) :
        
        if cumulative == False :
            ax[i].plot(Y_pred[asset] * 100, color = colors[i], label = f"{asset} Fitted")
            ax[i].scatter(Y.index, Y[asset] * 100, color = "black", alpha = 0.3, label = "Actual")
        
            ax[i].set_ylabel(ylabel = "Daily Return (%)", fontsize = 12)
            ax[i].legend( )
        
        else :
            cum_returns_asset = (1 + Y[asset]).cumprod( ) - 1 
            cum_returns_ff = (1 + Y_pred[asset]).cumprod( ) - 1 
        
            ax[i].plot(cum_returns_asset * 100, color = colors[i], label = asset)
            ax[i].plot(cum_returns_ff * 100, color = "black", label = "Fitted")
        
            ax[i].set_ylabel(ylabel = "Cumulative Returns (%)", fontsize = 12)
            ax[i].legend( )
        
    plt.style.use("default")
    plt.show( )

In [ ]:
plot_fitted_vs_actual(returns[tickers], ff_returns[tickers])

In [ ]:
plot_fitted_vs_actual(returns[tickers], ff_returns[tickers], cumulative = True)

In [ ]:
plot_fitted_vs_actual(returns[portfs + ["MP"]], ff_returns[portfs + ["MP"]])

In [ ]:
plot_fitted_vs_actual(returns[portfs + ["MP"]], ff_returns[portfs + ["MP"]], cumulative = True)

In [ ]:
def plot_residuals_vs_fitted(Y, Y_pred) :
    """
    Arguments :
    - Y : DataFrame of actual daily returns 
    - Y_pred : DataFrame of daily returns predicted by French-Farma model 
    
    Returns :
    - None 
    
    Function :
    - Draws a scatter plot with predited daily returns on the x-axis and residuals on the y-axis for each asset 
    - Draws 2 horizontal lines for the 95% confidence interval of residual values for each asset 
    """
    residuals = Y - Y_pred 
    
    n_assets = len(Y.columns)
    fig , ax = plt.subplots(figsize = (12 , n_assets * 5), sharex = False, sharey = False, nrows = n_assets, ncols = 1)
    
    colors = sns.color_palette("tab10") + ["limegreen", "fuchsia"]
    for i, asset in enumerate(Y.columns) :
        
        ax[i].scatter(Y_pred[asset] * 100, residuals[asset] * 100, color = colors[i], label = asset)
        
        ax[i].axhline(y = 0, color = "black", linestyle = "--")
        ax[i].axhline(y = (residuals[asset].mean( ) - 2 * residuals[asset].std( )) * 100, 
                      color = "red", linestyle = "--", label = "μ - 2σ (5%)")
        ax[i].axhline(y = (residuals[asset].mean( ) + 2 * residuals[asset].std( )) * 100, 
                      color = "green", linestyle = "--", label = "μ + 2σ (95%)")
        
        ax[i].set_xlabel(xlabel = "Fitted Return (%)", fontsize = 12)
        ax[i].set_ylabel(ylabel = "Residual (%)", fontsize = 12)
        ax[i].legend( )
        
    plt.style.use("default")
    plt.show( )

In [ ]:
plot_residuals_vs_fitted(returns[tickers], ff_returns[tickers])

In [ ]:
plot_residuals_vs_fitted(returns[portfs + ["MP"]], ff_returns[portfs + ["MP"]])

## Thank You for Your Attention. =))

# Author 
[Khai Lap Vuong](https://www.linkedin.com/in/khai-lap-vuong/)

## Other Contributor(s) 
[Ruslan Goyenko](https://www.linkedin.com/in/russ-goyenko-24a5777/)